In [1]:
# encoding=utf-8
import os.path as osp
import os
import copy
import matplotlib.pyplot as plt
import torch
from torch.nn import Linear
from sklearn.metrics import average_precision_score, roc_auc_score
from torch_geometric.data import TemporalData

from torch_geometric.nn import TGNMemory, TransformerConv
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.models.tgn import (LastNeighborLoader, IdentityMessage, MeanAggregator,
                                           LastAggregator)
from torch_geometric import *
from torch_geometric.utils import negative_sampling

from tqdm import tqdm
# from .autonotebook import tqdm as notebook_tqdm

import networkx as nx
import numpy as np
import math
import copy
import re
import time
import json
import pandas as pd
from random import choice
import gc
from graphviz import Digraph
import xxhash

from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time

from sklearn.metrics import confusion_matrix
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load utility methods
def hashgen(l):
    """Generate a single hash value from a list. @l is a list of
    string values, which can be properties of a node/edge. This
    function returns a single hashed integer value."""
    hasher = xxhash.xxh64()
    for e in l:
        hasher.update(e)
    return hasher.intdigest()


def datetime_to_ns_time(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    date,ns=date.split('.')

    timeArray = time.strptime(date, '%Y-%m-%dT%H:%M:%S')
    timeStamp = int(time.mktime(timeArray))
    timeStamp = timeStamp * 1000000000
    timeStamp += int(ns.split('Z')[0])
    return timeStamp


def datetime_to_timestamp_US(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    date=date.replace('-04:00','')
    if '.' in date:
        date,ms=date.split('.')
    else:
        ms=0
    tz = pytz.timezone('Etc/GMT+4')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp*1000+int(ms)
    return int(timeStamp)


def timestamp_to_datetime_US(ns):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    #ns = int(ns)
    tz = pytz.timezone('US/Eastern')
    ms=ns%1000
    ns//=1000
    dt = pytz.datetime.datetime.fromtimestamp(int(ns), tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s+='.'+str(ms)
#     s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    #print(f'T = {ns} {type(ns)} {s}')
    return s

pid_split_symble="#_"

host_split_symble="_@"



import xxhash

def tensor_find(t,x):
    t_np=t.numpy()
    idx=np.argwhere(t_np==x)
    return idx[0][0]+1


def std(t):
    t = np.array(t)
    return np.std(t)


def var(t):
    t = np.array(t)
    return np.var(t)


def mean(t):
    t = np.array(t)
    return np.mean(t)

def hashgen(l):
    """Generate a single hash value from a list. @l is a list of
    string values, which can be properties of a node/edge. This
    function returns a single hashed integer value."""
    hasher = xxhash.xxh64()
    for e in l:
        hasher.update(e)
    return hasher.intdigest()


def cal_pos_edges_loss(link_pred_ratio):
    loss=[]
    for i in link_pred_ratio:
        loss.append(criterion(i,torch.ones(1)))
    return torch.tensor(loss)

def cal_pos_edges_loss_multiclass(link_pred_ratio,labels):
    loss=[] 
    for i in range(len(link_pred_ratio)):
        loss.append(criterion(link_pred_ratio[i].reshape(1,-1),labels[i].reshape(-1)))
    return torch.tensor(loss)

# Connect to database

In [3]:
import psycopg2

from psycopg2 import extras as ex
# Create a postgreSQL DB connection object for storing provenance graph edges into DB
# Original '/var/run/postgresql/' has been replaced with 'localhost' since we are using docker and accessing as a service in port 5437
connect = psycopg2.connect(database = 'optc_db',
                           host = 'localhost',
                           user = 'postgres',
                           password = 'postgres',
                           port = '5437'
                          )

cur = connect.cursor()

# Load data

In [4]:
# Loading benign dataset/temporal graphs
# This graph not preprocessed, likely pre-processing bug
#graph_9_22_h201=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_22_host=SysClient0201_datalabel=benign.TemporalData")
graph_9_22_h402=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_22_host=SysClient0402_datalabel=benign.TemporalData")
graph_9_22_h660=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_22_host=SysClient0660_datalabel=benign.TemporalData")
graph_9_22_h501=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_22_host=SysClient0501_datalabel=benign.TemporalData")
graph_9_22_h051=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_22_host=SysClient0051_datalabel=benign.TemporalData")
graph_9_22_h209=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_22_host=SysClient0209_datalabel=benign.TemporalData")

In [5]:
train_data=graph_9_22_h660

## Testing data

In [6]:
# Load testing dataset/temporal graphs

graph_9_23_h201=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_23_host=SysClient0201_datalabel=evaluation.TemporalData")
graph_9_23_h402=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_23_host=SysClient0402_datalabel=evaluation.TemporalData")
graph_9_23_h660=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_23_host=SysClient0660_datalabel=evaluation.TemporalData")
graph_9_23_h501=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_23_host=SysClient0501_datalabel=evaluation.TemporalData")
graph_9_23_h051=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_23_host=SysClient0051_datalabel=evaluation.TemporalData")
graph_9_23_h207=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_23_host=SysClient0207_datalabel=evaluation.TemporalData")


graph_9_24_h201=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_24_host=SysClient0201_datalabel=evaluation.TemporalData")
graph_9_24_h402=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_24_host=SysClient0402_datalabel=evaluation.TemporalData")
graph_9_24_h660=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_24_host=SysClient0660_datalabel=evaluation.TemporalData")
graph_9_24_h501=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_24_host=SysClient0501_datalabel=evaluation.TemporalData")
graph_9_24_h051=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_24_host=SysClient0051_datalabel=evaluation.TemporalData")
graph_9_24_h207=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_24_host=SysClient0207_datalabel=evaluation.TemporalData")


graph_9_25_h201=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_25_host=SysClient0201_datalabel=evaluation.TemporalData")
graph_9_25_h402=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_25_host=SysClient0402_datalabel=evaluation.TemporalData")
graph_9_25_h660=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_25_host=SysClient0660_datalabel=evaluation.TemporalData")
graph_9_25_h501=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_25_host=SysClient0501_datalabel=evaluation.TemporalData")
graph_9_25_h051=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_25_host=SysClient0051_datalabel=evaluation.TemporalData")
graph_9_25_h207=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9_25_host=SysClient0207_datalabel=evaluation.TemporalData")



In [23]:
train_data=graph_9_25_h207

In [7]:
# TODO : investigate, authors didn't preprocessed this files why?
#  graph_9_19=torch.load("/home/shahidul2k9/data/optc/out/benign/9-19-h201.TemporalData")
# graph_9_20=torch.load("/home/shahidul2k9/data/optc/out/benign/9-20-h201.TemporalData")
# graph_9_21=torch.load("/home/shahidul2k9/data/optc/out/benign/9-21-h201.TemporalData")
# graph_9_22=torch.load("/home/shahidul2k9/data/optc/out/benign/9-22-h201.TemporalData")
# graph_9_23=torch.load("/home/shahidul2k9/data/optc/out/evaluation/9-23-h201.TemporalData")

#train_data=graph_9_22

# Generate node2msg

In [8]:
# Constructing the map for nodeid to msg
sql="select * from nodeid2msg;"
cur.execute(sql)
rows = cur.fetchall()

node_uuid2path={}  # nodeid => msg and node hash => nodeid
for i in tqdm(rows):
    node_uuid2path[i[0]]=i[1]

100%|██████████| 18965643/18965643 [00:09<00:00, 1942777.42it/s]


In [9]:
# The node2index of training data
# TODO : investigate, authors didn't preprocessed this files why?
#node_uuid2index_9_17_h201=torch.load("node_uuid2index_9_17_host=SysClient0201_datalabel=benign")
#node_uuid2index_9_18_h201=torch.load("node_uuid2index_9_18_host=SysClient0201_datalabel=benign")
#node_uuid2index_9_19_h201=torch.load("node_uuid2index_9_19_host=SysClient0201_datalabel=benign")
#node_uuid2index_9_20_h201=torch.load("node_uuid2index_9_20_host=SysClient0201_datalabel=benign")
#node_uuid2index_9_21_h201=torch.load("node_uuid2index_9_21_host=SysClient0201_datalabel=benign")
#node_uuid2index_9_22_h201=torch.load("node_uuid2index_9_22_host=SysClient0201_datalabel=benign")
#node_uuid2index_9_23_h201=torch.load("node_uuid2index_9_23_host=SysClient0201_datalabel=benign")

In [7]:
# TODO : investigate, authors didn't preprocessed this files why?
#node_uuid2index_9_22_h201=torch.load("node_uuid2index_9_22_host=SysClient0201_datalabel=benign")
node_uuid2index_9_22_h402=torch.load("node_uuid2index_9_22_host=SysClient0402_datalabel=benign")
node_uuid2index_9_22_h660=torch.load("node_uuid2index_9_22_host=SysClient0660_datalabel=benign")
node_uuid2index_9_22_h501=torch.load("node_uuid2index_9_22_host=SysClient0501_datalabel=benign")
node_uuid2index_9_22_h051=torch.load("node_uuid2index_9_22_host=SysClient0051_datalabel=benign")
node_uuid2index_9_22_h209=torch.load("node_uuid2index_9_22_host=SysClient0209_datalabel=benign")

In [8]:
# The node2index of testing data
node_uuid2index_9_23_h201=torch.load("node_uuid2index_9_23_host=SysClient0201_datalabel=evaluation")
node_uuid2index_9_24_h201=torch.load("node_uuid2index_9_24_host=SysClient0201_datalabel=evaluation")
node_uuid2index_9_25_h201=torch.load("node_uuid2index_9_25_host=SysClient0201_datalabel=evaluation")

node_uuid2index_9_23_h402=torch.load("node_uuid2index_9_23_host=SysClient0402_datalabel=evaluation")
node_uuid2index_9_24_h402=torch.load("node_uuid2index_9_24_host=SysClient0402_datalabel=evaluation")
node_uuid2index_9_25_h402=torch.load("node_uuid2index_9_25_host=SysClient0402_datalabel=evaluation")

node_uuid2index_9_23_h660=torch.load("node_uuid2index_9_23_host=SysClient0660_datalabel=evaluation")
node_uuid2index_9_24_h660=torch.load("node_uuid2index_9_24_host=SysClient0660_datalabel=evaluation")
node_uuid2index_9_25_h660=torch.load("node_uuid2index_9_25_host=SysClient0660_datalabel=evaluation")

node_uuid2index_9_23_h501=torch.load("node_uuid2index_9_23_host=SysClient0501_datalabel=evaluation")
node_uuid2index_9_24_h501=torch.load("node_uuid2index_9_24_host=SysClient0501_datalabel=evaluation")
node_uuid2index_9_25_h501=torch.load("node_uuid2index_9_25_host=SysClient0501_datalabel=evaluation")

node_uuid2index_9_23_h051=torch.load("node_uuid2index_9_23_host=SysClient0051_datalabel=evaluation")
node_uuid2index_9_24_h051=torch.load("node_uuid2index_9_24_host=SysClient0051_datalabel=evaluation")
node_uuid2index_9_25_h051=torch.load("node_uuid2index_9_25_host=SysClient0051_datalabel=evaluation")

node_uuid2index_9_23_h207=torch.load("node_uuid2index_9_23_host=SysClient0207_datalabel=evaluation")
node_uuid2index_9_24_h207=torch.load("node_uuid2index_9_24_host=SysClient0207_datalabel=evaluation")
node_uuid2index_9_25_h207=torch.load("node_uuid2index_9_25_host=SysClient0207_datalabel=evaluation")

In [9]:
# TODO : investigate, authors didn't preprocessed this files why?
# Calculate maximum possible node index over all the temporal graphs
maxnode_num=max(
    #len(node_uuid2index_9_17_h201)//2+1,
    # len(node_uuid2index_9_18_h201)//2+1,
    # len(node_uuid2index_9_19_h201)//2+1,
    # len(node_uuid2index_9_20_h201)//2+1,
    # len(node_uuid2index_9_21_h201)//2+1,
    # len(node_uuid2index_9_22_h201)//2+1,
    # len(node_uuid2index_9_23_h201)//2+1,
    
    # len(node_uuid2index_9_22_h201)//2+1,
    len(node_uuid2index_9_22_h402)//2+1,
    len(node_uuid2index_9_22_h660)//2+1,
    len(node_uuid2index_9_22_h501)//2+1,
    len(node_uuid2index_9_22_h051)//2+1,
    len(node_uuid2index_9_22_h209)//2+1,
    
    len(node_uuid2index_9_23_h201)//2+1,
    len(node_uuid2index_9_24_h201)//2+1,
    len(node_uuid2index_9_25_h201)//2+1,
    len(node_uuid2index_9_23_h402)//2+1,
    len(node_uuid2index_9_24_h402)//2+1,
    len(node_uuid2index_9_25_h402)//2+1,    
    
    len(node_uuid2index_9_23_h660)//2+1,
    len(node_uuid2index_9_24_h660)//2+1,
    len(node_uuid2index_9_25_h660)//2+1,
    
    len(node_uuid2index_9_23_h501)//2+1,
    len(node_uuid2index_9_24_h501)//2+1,
    len(node_uuid2index_9_25_h501)//2+1,
    
    len(node_uuid2index_9_23_h051)//2+1,
    len(node_uuid2index_9_24_h051)//2+1,
    len(node_uuid2index_9_25_h051)//2+1,
    
    len(node_uuid2index_9_23_h207)//2+1,
    len(node_uuid2index_9_24_h207)//2+1,
    len(node_uuid2index_9_25_h207)//2+1,    
)

In [10]:
# Log max node index over all graphs
maxnode_num

691859

In [11]:
# Edge <==> Index mapping
rel2id={1: 'OPEN',
 'OPEN': 1,
 2: 'READ',
 'READ': 2,
 3: 'CREATE',
 'CREATE': 3,
 4: 'MESSAGE',
 'MESSAGE': 4,
 5: 'MODIFY',
 'MODIFY': 5,
 6: 'START',
 'START': 6,
 7: 'RENAME',
 'RENAME': 7,
 8: 'DELETE',
 'DELETE': 8,
 9: 'TERMINATE',
 'TERMINATE': 9,
 10: 'WRITE',
 'WRITE': 10}

# Setting the parameters and Training

In [12]:
# Set embedding dimensions and other parameters
time_dim = 100
edge_embedding_dimension=100
embedding_dim = edge_embedding_dimension
neighbor_size=20
memory_dim = 100

max_node_num = maxnode_num+2
min_dst_idx, max_dst_idx = 0, max_node_num
# Set neighbour loader
neighbor_loader = LastNeighborLoader(max_node_num, size=neighbor_size, device=device)

In [13]:
# Set batch size
BATCH=1024
# Configure graph attention layer
class GraphAttentionEmbedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super(GraphAttentionEmbedding, self).__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        # Set two succesive convolution filters
        self.conv = TransformerConv(in_channels, out_channels, heads=8,
                                    dropout=0.0, edge_dim=edge_dim)
        self.conv2 = TransformerConv(out_channels*8, out_channels,heads=1, concat=False,
                             dropout=0.0, edge_dim=edge_dim)

    def forward(self, x, last_update, edge_index, t, msg):
        last_update.to(device)
        x = x.to(device)
        t = t.to(device)
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        x = F.relu(self.conv(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        return x

# Configure Link Predictor
class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(LinkPredictor, self).__init__()
        self.lin_src = Linear(in_channels, in_channels*2)
        self.lin_dst = Linear(in_channels, in_channels*2)
        self.lin_seq = nn.Sequential(
            Linear(in_channels * 4, in_channels * 8),
            torch.nn.BatchNorm1d(in_channels * 8),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels * 8, in_channels * 2),
            torch.nn.BatchNorm1d(in_channels * 2),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels * 2, int(in_channels // 2)),
            torch.nn.BatchNorm1d(int(in_channels // 2)),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(int(in_channels // 2), train_data.msg.shape[1] - 32)
        )

    def forward(self, z_src, z_dst):
        h = torch.cat([self.lin_src(z_src) , self.lin_dst(z_dst)],dim=-1)      
        h = self.lin_seq (h)        
        return h
# Configure memory layer
memory = TGNMemory(
    max_node_num,
    train_data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(train_data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)
# Create graph attention embedding layer
gnn = GraphAttentionEmbedding(
    in_channels=memory_dim,
    out_channels=embedding_dim,
    msg_dim=train_data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

# Create link predictor
link_pred = LinkPredictor(in_channels=embedding_dim).to(device)

# Set Adam optimizer
optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters())
    | set(link_pred.parameters()), lr=0.00005, eps=1e-08,weight_decay=0.01)

criterion = nn.CrossEntropyLoss()


assoc = torch.empty(max_node_num, dtype=torch.long, device=device)


saved_nodes=set()

BATCH=1024
# Training method
def train(train_data):

    # Signal each layer to prepares for training modes
    memory.train()
    gnn.train()
    link_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.
    saved_nodes=set()

    total_loss = 0
    # Iterate over the batches and train the models
    for batch in train_data.seq_batches(batch_size=BATCH):
        # Reset the gradiant of all parameters to zeroes
        optimizer.zero_grad()

        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg        

        n_id = torch.cat([src, pos_dst]).unique()

        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = memory(n_id)

        z = gnn(z, last_update, edge_index, train_data.t[e_id], train_data.msg[e_id])

        # Predict likelihood of positive edge
        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])       

        y_pred = torch.cat([pos_out], dim=0)

        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l)           

        y_true = torch.tensor(y_true)
        y_true=y_true.reshape(-1).to(torch.long)

        loss = criterion(y_pred, y_true)     

        # Update memory and neighbor loader with ground-truth state.
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)       
        # Computes the gradients of the loss with respect to the model's parameters
        loss.backward()
        # Updates the model parameters using the computed gradients
        optimizer.step()
        # Detaches the memory tensor from the computation graph
        memory.detach()
        # sum losses at this epoch
        total_loss += float(loss) * batch.num_events
    return total_loss / train_data.num_events



In [17]:
# list out training graphs
train_graphs=[
    # graph_9_22_h201,
    graph_9_22_h402,
    graph_9_22_h660,
    graph_9_22_h501,
    graph_9_22_h051,
    graph_9_22_h209,
]
print(f"{embedding_dim=}")
print(f"{gnn=}")
# Train the model
for epoch in tqdm(range(1, 11)):
    for g in train_graphs:
        loss = train(g)
        print(f'  Epoch: {epoch:02d}, Loss: {loss:.4f}')


memory.reset_state()  # Start with a fresxh memory.
neighbor_loader.reset_state() 
model=[memory,gnn, link_pred,neighbor_loader]
# Save the model
torch.save(model,f"/home/shahidul2k9/data/optc/model/model_saved_traindata=hosts_9_22.pt")

embedding_dim=100
gnn=GraphAttentionEmbedding(
  (time_enc): TimeEncoder(
    (lin): Linear(in_features=1, out_features=100, bias=True)
  )
  (conv): TransformerConv(100, 100, heads=8)
  (conv2): TransformerConv(800, 100, heads=1)
)


  0%|          | 0/10 [00:00<?, ?it/s]

  Epoch: 01, Loss: 1.4748
  Epoch: 01, Loss: 0.9420
  Epoch: 01, Loss: 0.8856
  Epoch: 01, Loss: 0.8082


 10%|█         | 1/10 [59:15<8:53:19, 3555.48s/it]

  Epoch: 01, Loss: 0.7344
  Epoch: 02, Loss: 0.8230
  Epoch: 02, Loss: 0.6913
  Epoch: 02, Loss: 0.7675
  Epoch: 02, Loss: 0.7262


...

  Epoch: 08, Loss: 0.6508
  Epoch: 09, Loss: 0.7235
  Epoch: 09, Loss: 0.6405
  Epoch: 09, Loss: 0.7095
  Epoch: 09, Loss: 0.6827


 90%|█████████ | 9/10 [8:37:56<57:05, 3425.19s/it]  

  Epoch: 09, Loss: 0.6496
  Epoch: 10, Loss: 0.7215
  Epoch: 10, Loss: 0.6392
  Epoch: 10, Loss: 0.7086
  Epoch: 10, Loss: 0.6817


100%|██████████| 10/10 [9:34:00<00:00, 3444.10s/it]

  Epoch: 10, Loss: 0.6487


# Test

## Define the function for testing

In [14]:
import time 

@torch.no_grad()
def test_day_new(inference_data,path,nodeuuid2index):
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
    # Switch each layer from training to test modes
    memory.eval()
    gnn.eval()
    link_pred.eval()
    
    memory.reset_state()  
    neighbor_loader.reset_state()  
    
    time_with_loss={}
    total_loss = 0    
    edge_list=[]
    
    unique_nodes=torch.tensor([])
    total_edges=0
    


    start_time=int(inference_data.t[0])
    event_count=0
    
    pos_o=[]
    
    loss_list=[]


    print("after merge:",inference_data)
    

    start = time.perf_counter()
    # Iterate over batches of validation dataset
    for batch in inference_data.seq_batches(batch_size=BATCH):
        
        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg
        unique_nodes=torch.cat([unique_nodes,src,pos_dst]).unique()
        total_edges+=BATCH
        
       
        n_id = torch.cat([src, pos_dst]).unique()       
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)

        # Pass the node representations and graph information through the Graph Neural Network (GNN)
        z = gnn(z, last_update, edge_index, inference_data.t[e_id], inference_data.msg[e_id])

        # Predict the likelihood of an edge existing between the source and positive destination nodes
        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])
        
        pos_o.append(pos_out)
        y_pred = torch.cat([pos_out], dim=0)

        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l) 
        y_true = torch.tensor(y_true)
        y_true=y_true.reshape(-1).to(torch.long)
        
        # Compute the loss for positive link predictions
        loss = criterion(y_pred, y_true)

        # Accumulate the loss, weighted by the number of events in the batch
        total_loss += float(loss) * batch.num_events
     
        # Update memory state
        memory.update_state(src, pos_dst, t, msg)
        # Insert the edge into the neighbor loader for future neighborhood sampling
        neighbor_loader.insert(src, pos_dst)
        

        each_edge_loss= cal_pos_edges_loss_multiclass(pos_out,y_true)
        
        for i in range(len(pos_out)):
            srcnode= int(src[i])
            dstnode=  int(pos_dst[i])
            
            srcmsg=str(nodeuuid2index[srcnode]) 
            dstmsg=str(nodeuuid2index[dstnode])
            t_var=int(t[i])
            edgeindex=tensor_find(msg[i][16:-16],1) 
            edge_type=rel2id[edgeindex]
            loss=each_edge_loss[i]    

            temp_dic={}
            temp_dic['loss']=float(loss)
            temp_dic['srcnode']=srcnode
            temp_dic['dstnode']=dstnode
            temp_dic['srcmsg']=srcmsg
            temp_dic['dstmsg']=dstmsg
            temp_dic['edge_type']=edge_type
            temp_dic['time']=t_var

            edge_list.append(temp_dic)
        
        event_count+=len(batch.src)
        if t[-1]>start_time+60000*15:

            time_interval=timestamp_to_datetime_US(start_time)+"~"+timestamp_to_datetime_US(int(t[-1]))

            end = time.perf_counter()
            time_with_loss[time_interval]={'loss':loss,
                                
                                          'nodes_count':len(unique_nodes),
                                          'total_edges':total_edges,
                                          'costed_time':(end-start)}
            
            
            log=open(path+"/"+time_interval+".txt",'w')

            
            for e in edge_list: 
                loss+=e['loss']

            loss=loss/event_count   
            print(f'Time: {time_interval}, Loss: {loss:.4f}, Nodes_count: {len(unique_nodes)}, Cost Time: {(end-start):.2f}s')
            edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True)  
            for e in edge_list: 
                log.write(str(e))
                log.write("\n") 
            event_count=0
            total_loss=0
            loss=0
            start_time=int(t[-1])
            log.close()
            edge_list.clear()
            
 
    return time_with_loss


## 9-22 hosts

In [15]:
# load the the trained model
model=torch.load("/home/shahidul2k9/data/optc/model/model_saved_traindata=hosts_9_22.pt")
memory,gnn, link_pred,neighbor_loader=model

In [34]:
# TODO : investigate, authors didn't preprocessed this files why?
#ans_9_22_h201=test_day_new(graph_9_22_h201,"graph_9_22_h201",node_uuid2index_9_22_h201)

In [35]:
ans_9_22_h402=test_day_new(graph_9_22_h402,"/home/shahidul2k9/data/optc/test/graph_9_22_h402",node_uuid2index_9_22_h402)

after merge: TemporalData(dst=[4407209], msg=[4407209, 42], src=[4407209], t=[4407209])
Time: 2019-09-22 00:00:00.8~2019-09-22 00:15:00.418, Loss: 0.8266, Nodes_count: 9470, Cost Time: 5.48s
Time: 2019-09-22 00:15:00.418~2019-09-22 00:30:42.861, Loss: 0.5816, Nodes_count: 15043, Cost Time: 7.99s
Time: 2019-09-22 00:30:42.861~2019-09-22 00:45:44.758, Loss: 0.9821, Nodes_count: 28193, Cost Time: 13.21s
Time: 2019-09-22 00:45:44.758~2019-09-22 01:01:20.483, Loss: 0.5469, Nodes_count: 33751, Cost Time: 18.90s
Time: 2019-09-22 01:01:20.483~2019-09-22 01:16:20.612, Loss: 0.7375, Nodes_count: 42300, Cost Time: 26.96s
Time: 2019-09-22 01:16:20.612~2019-09-22 01:32:01.859, Loss: 0.6598, Nodes_count: 47591, Cost Time: 30.24s
Time: 2019-09-22 01:32:01.859~2019-09-22 01:47:09.362, Loss: 0.6077, Nodes_count: 53575, Cost Time: 35.27s
Time: 2019-09-22 01:47:09.362~2019-09-22 02:02:34.719, Loss: 0.6372, Nodes_count: 60981, Cost Time: 42.71s
Time: 2019-09-22 02:02:34.719~2019-09-22 02:17:40.622, Loss: 

In [36]:
ans_9_22_h660=test_day_new(graph_9_22_h660,"/home/shahidul2k9/data/optc/test/graph_9_22_h660",node_uuid2index_9_22_h660)

after merge: TemporalData(dst=[3886983], msg=[3886983, 42], src=[3886983], t=[3886983])
Time: 2019-09-22 00:00:00.61~2019-09-22 00:15:05.35, Loss: 0.8379, Nodes_count: 8706, Cost Time: 4.20s
Time: 2019-09-22 00:15:05.35~2019-09-22 00:30:36.758, Loss: 0.6762, Nodes_count: 14202, Cost Time: 6.64s
Time: 2019-09-22 00:30:36.758~2019-09-22 00:45:45.546, Loss: 0.8503, Nodes_count: 23391, Cost Time: 10.93s
Time: 2019-09-22 00:45:45.546~2019-09-22 01:01:09.871, Loss: 0.6899, Nodes_count: 28969, Cost Time: 16.40s
Time: 2019-09-22 01:01:09.871~2019-09-22 01:16:25.366, Loss: 0.6787, Nodes_count: 35428, Cost Time: 22.36s
Time: 2019-09-22 01:16:25.366~2019-09-22 01:32:01.901, Loss: 0.6835, Nodes_count: 40339, Cost Time: 25.20s
Time: 2019-09-22 01:32:01.901~2019-09-22 01:47:03.66, Loss: 0.8264, Nodes_count: 48623, Cost Time: 29.76s
Time: 2019-09-22 01:47:03.66~2019-09-22 02:02:20.609, Loss: 0.7055, Nodes_count: 54441, Cost Time: 35.30s
Time: 2019-09-22 02:02:20.609~2019-09-22 02:17:29.185, Loss: 0.6

In [37]:
ans_9_22_h501=test_day_new(graph_9_22_h501,"/home/shahidul2k9/data/optc/test/graph_9_22_h501",node_uuid2index_9_22_h501)

after merge: TemporalData(dst=[4237267], msg=[4237267, 42], src=[4237267], t=[4237267])
Time: 2019-09-22 00:00:00.24~2019-09-22 00:15:04.496, Loss: 0.8670, Nodes_count: 12073, Cost Time: 5.85s
Time: 2019-09-22 00:15:04.496~2019-09-22 00:30:16.369, Loss: 1.2992, Nodes_count: 29016, Cost Time: 11.80s
Time: 2019-09-22 00:30:16.369~2019-09-22 00:45:17.879, Loss: 0.6959, Nodes_count: 35650, Cost Time: 15.23s
Time: 2019-09-22 00:45:17.879~2019-09-22 01:00:47.878, Loss: 0.5800, Nodes_count: 41287, Cost Time: 21.32s
Time: 2019-09-22 01:00:47.878~2019-09-22 01:15:52.8, Loss: 0.9626, Nodes_count: 54465, Cost Time: 29.98s
Time: 2019-09-22 01:15:52.8~2019-09-22 01:31:49.443, Loss: 0.6243, Nodes_count: 59407, Cost Time: 33.43s
Time: 2019-09-22 01:31:49.443~2019-09-22 01:46:51.671, Loss: 0.6761, Nodes_count: 65495, Cost Time: 38.11s
Time: 2019-09-22 01:46:51.671~2019-09-22 02:02:00.604, Loss: 0.6505, Nodes_count: 71595, Cost Time: 44.84s
Time: 2019-09-22 02:02:00.604~2019-09-22 02:17:02.361, Loss: 0

In [38]:
ans_9_22_h051=test_day_new(graph_9_22_h051,"/home/shahidul2k9/data/optc/test/graph_9_22_h051",node_uuid2index_9_22_h051)

after merge: TemporalData(dst=[4072257], msg=[4072257, 42], src=[4072257], t=[4072257])
Time: 2019-09-22 00:00:00.259~2019-09-22 00:15:03.94, Loss: 0.9110, Nodes_count: 9852, Cost Time: 4.86s
Time: 2019-09-22 00:15:03.94~2019-09-22 00:30:49.979, Loss: 0.6415, Nodes_count: 15519, Cost Time: 7.87s
Time: 2019-09-22 00:30:49.979~2019-09-22 00:46:00.351, Loss: 0.7857, Nodes_count: 24029, Cost Time: 12.47s
Time: 2019-09-22 00:46:00.351~2019-09-22 01:01:09.92, Loss: 1.0906, Nodes_count: 35370, Cost Time: 20.37s
Time: 2019-09-22 01:01:09.92~2019-09-22 01:16:13.46, Loss: 0.5851, Nodes_count: 41378, Cost Time: 26.75s
Time: 2019-09-22 01:16:13.46~2019-09-22 01:31:16.802, Loss: 0.7048, Nodes_count: 46381, Cost Time: 29.83s
Time: 2019-09-22 01:31:16.802~2019-09-22 01:46:29.171, Loss: 0.8591, Nodes_count: 54687, Cost Time: 35.49s
Time: 2019-09-22 01:46:29.171~2019-09-22 02:02:00.957, Loss: 0.6051, Nodes_count: 60573, Cost Time: 41.78s
Time: 2019-09-22 02:02:00.957~2019-09-22 02:17:07.743, Loss: 0.60

In [39]:
ans_9_22_h209=test_day_new(graph_9_22_h209,"/home/shahidul2k9/data/optc/test/graph_9_22_h209",node_uuid2index_9_22_h209)

after merge: TemporalData(dst=[3850957], msg=[3850957, 42], src=[3850957], t=[3850957])
Time: 2019-09-22 00:00:00.7~2019-09-22 00:15:22.17, Loss: 0.7450, Nodes_count: 8863, Cost Time: 4.57s
Time: 2019-09-22 00:15:22.17~2019-09-22 00:30:43.655, Loss: 0.6284, Nodes_count: 14441, Cost Time: 7.10s
Time: 2019-09-22 00:30:43.655~2019-09-22 00:45:45.353, Loss: 0.6107, Nodes_count: 21181, Cost Time: 10.22s
Time: 2019-09-22 00:45:45.353~2019-09-22 01:01:09.708, Loss: 0.6381, Nodes_count: 28967, Cost Time: 16.68s
Time: 2019-09-22 01:01:09.708~2019-09-22 01:16:25.325, Loss: 0.5281, Nodes_count: 35094, Cost Time: 22.83s
Time: 2019-09-22 01:16:25.325~2019-09-22 01:31:58.219, Loss: 0.5433, Nodes_count: 39795, Cost Time: 25.58s
Time: 2019-09-22 01:31:58.219~2019-09-22 01:47:05.805, Loss: 0.5951, Nodes_count: 46028, Cost Time: 30.28s
Time: 2019-09-22 01:47:05.805~2019-09-22 02:03:02.186, Loss: 0.5158, Nodes_count: 51893, Cost Time: 35.73s
Time: 2019-09-22 02:03:02.186~2019-09-22 02:18:12.632, Loss: 0.

## 9-23~25 hosts

In [16]:
# load the models
model=torch.load("/home/shahidul2k9/data/optc/model/model_saved_traindata=hosts_9_22.pt")
memory,gnn, link_pred,neighbor_loader=model

In [40]:
# Test the input graphs with trained models
graph_9_23_h201=test_day_new(graph_9_23_h201,"/home/shahidul2k9/data/optc/test/graph_9_23_h201",node_uuid2index_9_23_h201)
graph_9_23_h402=test_day_new(graph_9_23_h402,"/home/shahidul2k9/data/optc/test/graph_9_23_h402",node_uuid2index_9_23_h402)
graph_9_23_h660=test_day_new(graph_9_23_h660,"/home/shahidul2k9/data/optc/test/graph_9_23_h660",node_uuid2index_9_23_h660)
graph_9_23_h501=test_day_new(graph_9_23_h501,"/home/shahidul2k9/data/optc/test/graph_9_23_h501",node_uuid2index_9_23_h501)
graph_9_23_h051=test_day_new(graph_9_23_h051,"/home/shahidul2k9/data/optc/test/graph_9_23_h051",node_uuid2index_9_23_h051)
graph_9_23_h207=test_day_new(graph_9_23_h207,"/home/shahidul2k9/data/optc/test/graph_9_23_h207",node_uuid2index_9_23_h207)

graph_9_24_h201=test_day_new(graph_9_24_h201,"/home/shahidul2k9/data/optc/test/graph_9_24_h201",node_uuid2index_9_24_h201)
graph_9_24_h402=test_day_new(graph_9_24_h402,"/home/shahidul2k9/data/optc/test/graph_9_24_h402",node_uuid2index_9_24_h402)
graph_9_24_h660=test_day_new(graph_9_24_h660,"/home/shahidul2k9/data/optc/test/graph_9_24_h660",node_uuid2index_9_24_h660)
graph_9_24_h501=test_day_new(graph_9_24_h501,"/home/shahidul2k9/data/optc/test/graph_9_24_h501",node_uuid2index_9_24_h501)
graph_9_24_h051=test_day_new(graph_9_24_h051,"/home/shahidul2k9/data/optc/test/graph_9_24_h051",node_uuid2index_9_24_h051)
graph_9_24_h207=test_day_new(graph_9_24_h207,"/home/shahidul2k9/data/optc/test/graph_9_24_h207",node_uuid2index_9_24_h207)

graph_9_25_h201=test_day_new(graph_9_25_h201,"/home/shahidul2k9/data/optc/test/graph_9_25_h201",node_uuid2index_9_25_h201)
graph_9_25_h402=test_day_new(graph_9_25_h402,"/home/shahidul2k9/data/optc/test/graph_9_25_h402",node_uuid2index_9_25_h402)
graph_9_25_h660=test_day_new(graph_9_25_h660,"/home/shahidul2k9/data/optc/test/graph_9_25_h660",node_uuid2index_9_25_h660)
graph_9_25_h501=test_day_new(graph_9_25_h501,"/home/shahidul2k9/data/optc/test/graph_9_25_h501",node_uuid2index_9_25_h501)
graph_9_25_h051=test_day_new(graph_9_25_h051,"/home/shahidul2k9/data/optc/test/graph_9_25_h051",node_uuid2index_9_25_h051)
graph_9_25_h207=test_day_new(graph_9_25_h207,"/home/shahidul2k9/data/optc/test/graph_9_25_h207",node_uuid2index_9_25_h207)

after merge: TemporalData(dst=[2349592], msg=[2349592, 42], src=[2349592], t=[2349592])
Time: 2019-09-23 09:06:13.9~2019-09-23 09:21:19.885, Loss: 0.8075, Nodes_count: 7768, Cost Time: 3.48s
Time: 2019-09-23 09:21:19.885~2019-09-23 09:36:28.932, Loss: 0.6553, Nodes_count: 16348, Cost Time: 6.77s
Time: 2019-09-23 09:36:28.932~2019-09-23 09:51:40.187, Loss: 0.6434, Nodes_count: 24639, Cost Time: 13.19s
Time: 2019-09-23 09:51:40.187~2019-09-23 10:06:52.508, Loss: 0.4964, Nodes_count: 31436, Cost Time: 23.34s
Time: 2019-09-23 10:06:52.508~2019-09-23 10:21:54.256, Loss: 0.5641, Nodes_count: 40544, Cost Time: 30.21s
Time: 2019-09-23 10:21:54.256~2019-09-23 10:37:17.566, Loss: 0.6078, Nodes_count: 47000, Cost Time: 33.63s
Time: 2019-09-23 10:37:17.566~2019-09-23 10:52:41.476, Loss: 0.4974, Nodes_count: 51987, Cost Time: 39.53s
Time: 2019-09-23 10:52:41.476~2019-09-23 11:08:03.86, Loss: 0.6301, Nodes_count: 60838, Cost Time: 48.77s
Time: 2019-09-23 11:08:03.86~2019-09-23 11:23:05.136, Loss: 0.

KeyboardInterrupt: 

In [ ]:
# Save the tested graphs
torch.save(graph_9_23_h201,"/home/shahidul2k9/data/optc/test_res/graph_9_23_h201")
torch.save(graph_9_23_h402,"/home/shahidul2k9/data/optc/test_res/graph_9_23_h402")
torch.save(graph_9_23_h660,"/home/shahidul2k9/data/optc/test_res/graph_9_23_h660")
torch.save(graph_9_23_h501,"/home/shahidul2k9/data/optc/test_res/graph_9_23_h501")
torch.save(graph_9_23_h051,"/home/shahidul2k9/data/optc/test_res/graph_9_23_h051")
torch.save(graph_9_23_h207,"/home/shahidul2k9/data/optc/test_res/graph_9_23_h207")

torch.save(graph_9_24_h201,"/home/shahidul2k9/data/optc/test_res/graph_9_24_h201")
torch.save(graph_9_24_h402,"/home/shahidul2k9/data/optc/test_res/graph_9_24_h402")
torch.save(graph_9_24_h660,"/home/shahidul2k9/data/optc/test_res/graph_9_24_h660")
torch.save(graph_9_24_h501,"/home/shahidul2k9/data/optc/test_res/graph_9_24_h501")
torch.save(graph_9_24_h051,"/home/shahidul2k9/data/optc/test_res/graph_9_24_h051")
torch.save(graph_9_24_h207,"/home/shahidul2k9/data/optc/test_res/graph_9_24_h207")

torch.save(graph_9_25_h201,"/home/shahidul2k9/data/optc/test_res/graph_9_25_h201")
torch.save(graph_9_25_h402,"/home/shahidul2k9/data/optc/test_res/graph_9_25_h402")
torch.save(graph_9_25_h660,"/home/shahidul2k9/data/optc/test_res/graph_9_25_h660")
torch.save(graph_9_25_h501,"/home/shahidul2k9/data/optc/test_res/graph_9_25_h501")
torch.save(graph_9_25_h051,"/home/shahidul2k9/data/optc/test_res/graph_9_25_h051")
torch.save(graph_9_25_h207,"/home/shahidul2k9/data/optc/test_res/graph_9_25_h207")

# Compute the anomalous score

In [16]:
def cal_train_IDF(find_str,file_list):
    include_count=0
    for f_path in (file_list):
        f=open(f_path)
        if find_str in f.read():
            include_count+=1             
    IDF=math.log(len(file_list)/(include_count+1))
    return IDF


def cal_IDF(find_str,file_path,file_list):
    file_list=os.listdir(file_path)
    include_count=0
    different_neighbor=set()
    for f_path in (file_list):
        f=open(file_path+f_path)
        if find_str in f.read():
            include_count+=1
                
    IDF=math.log(len(file_list)/(include_count+1))
    
    return IDF,1


def cal_IDF_by_file_in_mem(find_str,file_list):
    include_count=0
    different_neighbor=set()
    for f in (file_list):       
        if find_str in f:
            include_count+=1
    IDF=math.log(len(file_list)/(include_count+1))    
    return IDF

def cal_redundant(find_str,edge_list):
    
    different_neighbor=set()
    for e in edge_list:
        if find_str in str(e):
            different_neighbor.add(e[0])
            different_neighbor.add(e[1])
    return len(different_neighbor)-2

def cal_anomaly_loss(loss_list,edge_list,file_path):
    
    if len(loss_list)!=len(edge_list):
        print("error!")
        return 0
    count=0
    loss_sum=0
    loss_std=std(loss_list)
    loss_mean=mean(loss_list)
    edge_set=set()
    node_set=set()
    node2redundant={}
    
    thr=loss_mean+2.5*loss_std

    print("thr:",thr)
    
    for i in range(len(loss_list)):
        if loss_list[i]>thr:
            count+=1
            src_node=edge_list[i][0]
            dst_node=edge_list[i][1]

            loss_sum+=loss_list[i]
    
            node_set.add(src_node)
            node_set.add(dst_node)
            edge_set.add(edge_list[i][0]+edge_list[i][1])
    return count, loss_sum/count,node_set,edge_set
#     return count, count/len(loss_list)

# Construct the relations between time windows

In [17]:
    
file_list=[]
# TODO : Investigate why missing in preprocessing steps
# file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h201/"
# file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h201/")
# for i in file_l:
#     file_list.append(file_path+i)

file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h402/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h402/")
for i in file_l:
    file_list.append(file_path+i)
    
file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h660/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h660/")
for i in file_l:
    file_list.append(file_path+i)
    
file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h501/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h501/")
for i in file_l:
    file_list.append(file_path+i)
    
file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h051/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h051/")
for i in file_l:
    file_list.append(file_path+i)    
    
    
file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h209/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h209/")
for i in file_l:
    file_list.append(file_path+i) 

    
def is_include_key_word(s):
    keywords=[
         '->',
        '.DLL',
        '.dll',
        '.dat', 
       '.DAT', 
        'CACHE',
        'Cache',
        '.docx',
        '.lnk',
        '.LNK',
        '.pptx',
        '.xlsx',
        'CVR',
        'cvr',
        'ZLEAZER',
        'zleazer',
        'SOFTWAREPROTECTIONPLATFORM',
        'documents',
        '.log',
        '.nls',
        '.EVTX',
        '.evtx',
        '.tmp',
        '.TMP',
        'Windows/Logs/',
        'Windows/system32/',
        'Windows/System32/',
        '/Temp/',
        'Users',
        'USERS',
        'Program Files',
        'WINDOWS',
        'Windows',
        '$SII',
        'svchost.exe',
        'gpscript.exe',
        'python.exe',
        'rundll32.exe',
        'consent.exe',
        'python27',
        'Python27',
      ]
    flag=False
    for i in keywords:
        if i in s:
            flag=True
    return flag    
    
    
def cal_set_rel(s1,s2):
    new_s=s1 & s2
    count=0
    for i in new_s:
        if is_include_key_word(i) is not True:

            if i in node_IDF.keys():
                IDF=node_IDF[i]
            else:
                #TODO: Investigate why file_list is 0, maybe due to missing file
                if len(file_list) > 0:
                    IDF = IDF=math.log(len(file_list)/(1))
                else:
                    IDF = float("inf") 
            if IDF>(math.log(len(file_list)*0.9)):
                print("node:",i," IDF:",IDF)
                count+=1
    return count

In [18]:
math.log(len(file_list)/(1))

6.150602768446279

# Compute the IDF-1

In [19]:


file_list=[]
# TODO: Investigate why missing in preprocessing steps
# file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h201/"
# file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h201/")
# for i in file_l:
#     file_list.append(file_path+i)

file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h402/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h402/")
for i in file_l:
    file_list.append(file_path+i)
    
file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h660/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h660/")
for i in file_l:
    file_list.append(file_path+i)
    
file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h501/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h501/")
for i in file_l:
    file_list.append(file_path+i)
    
file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h051/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h051/")
for i in file_l:
    file_list.append(file_path+i)    
    
    
file_path="/home/shahidul2k9/data/optc/test/graph_9_22_h209/"
file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22_h209/")
for i in file_l:
    file_list.append(file_path+i)    



In [ ]:
node_set=set()

for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        if jdata['loss']>0:
            if '->' not in str(jdata['srcmsg']):
                node_set.add(str(jdata['srcmsg']).split("_@")[-1])
            if '->' not in str(jdata['dstmsg']):
                node_set.add(str(jdata['dstmsg']).split("_@")[-1]) 


node_list=list(node_set)
del node_set


KeyboardInterrupt: 

In [21]:
files_mem_list=[]
for f_path in (file_list):
        f=open(f_path)
        files_mem_list.append(f.read())

In [20]:
def process_node_idf(_node_list,_files_mem_list,share_node_IDF):
    for n in tqdm(_node_list):  
        find_str=n
        IDF=cal_IDF_by_file_in_mem(n,_files_mem_list)
        share_node_IDF[n]=IDF

import multiprocessing as mp

In [108]:
share_node_IDF = mp.Manager().dict()
cores=28
offset=math.ceil(len(node_list)/cores)
node_list_split=[]
for i in range(0,len(node_list),offset):
    node_list_split+=[node_list[i:i+offset]]     
process_list=[]
for i in range(cores):
    process_list.append(mp.Process(target=process_node_idf, args=(node_list_split[i],files_mem_list,share_node_IDF)))
for i in process_list:
    i.start()

In [109]:
node_IDF=dict(share_node_IDF)
torch.save(node_IDF,"node_IDF_9_22_hosts")
print("IDF weight calculate complete!")

IDF weight calculate complete!


In [22]:
# force to terminate all process
for i in process_list:
    i.terminate()

NameError: name 'process_list' is not defined

In [111]:
# TODO: Investigate why missing this file in preprocessing and leaning stage. There is no save operation written by original authors
# # The IDF of the date 9-22 h201
# node_IDF={}
# node_set=set()

# file_list=[]

# file_path="/home/shahidul2k9/data/optc/test/graph_9_22/"
# file_l=os.listdir("/home/shahidul2k9/data/optc/test/graph_9_22/")


# for i in file_l:
#     file_list.append(file_path+i)

# for f_path in tqdm(file_list):
#     f=open(f_path)
#     for line in f:
#         l=line.strip()
#         jdata=eval(l)
#         if jdata['loss']>0:
#             if '->' not in str(jdata['srcmsg']):
#                 node_set.add(str(jdata['srcmsg']))
#             if '->' not in str(jdata['dstmsg']):
#                 node_set.add(str(jdata['dstmsg'])) 

# for n in tqdm(node_set):
# #     find_str=list(eval(n).values())[0]
#     IDF=cal_train_IDF(n,file_list)
#     node_IDF[n]=IDF


# torch.save(node_IDF,"node_IDF_9_22_without_netflow")
# print("IDF weight calculate complete!")

# labelling

In [23]:
labels={}
path="/home/shahidul2k9/data/optc/test/graph_9_23_h201/"
filelist = os.listdir(path)
filelist.sort()
for f in filelist:
    labels[path+f]=0


## h201

In [24]:
label_h201={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h201/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h201[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h201/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h201[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h201/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h201[test_path+f]=0

attack_list=[
'/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 11:23:44.136~2019-09-23 11:38:30.698.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 11:38:40.698~2019-09-23 11:53:39.57.txt',

 '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 12:38:24.95~2019-09-23 12:54:14.286.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 12:55:28.286~2019-09-23 13:09:50.95.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 13:10:24.95~2019-09-23 13:24:56.43.txt',

]

# for i in attack_list:
#     label_h201[i]=1


for i in label_h201:
    for k in attack_list:
        f = k.split('~')[-1]
        if i.split('~')[-1] == k.split('~')[-1]:
            label_h201[i] = 1
            break
print(f'Attack Files:  {len(attack_list)} - {sum(label_h201.values())}')

Attack Files:  5 - 5


## h402

In [25]:
label_h402={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h402/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h402[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h402/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h402[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h402/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h402[test_path+f]=0

attack_list=[
 '/home/shahidul2k9/data/optc/test/graph_9_23_h402/2019-09-23 13:10:24.429~2019-09-23 13:25:09.374.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h402/2019-09-23 13:25:20.374~2019-09-23 13:40:21.268.txt',

    # adjust
#      'graph_9_23_h402/2019-09-23 13:40:16.268~2019-09-23 13:55:31.31.txt',
#  'graph_9_23_h402/2019-09-23 13:55:12.31~2019-09-23 14:10:58.2.txt',
]
# for i in attack_list:
#     label_h402[i]=1

for i in label_h402:
    for k in attack_list:
        f = k.split('~')[-1]
        if i.split('~')[-1] == k.split('~')[-1]:
            label_h402[i] = 1
            break
print(f'Attack Files:  {len(attack_list)} - {sum(label_h402.values())}')

Attack Files:  2 - 2


## h660

In [26]:
label_h660={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h660/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h660[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h660/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h660[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h660/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h660[test_path+f]=0

attack_list=[
'/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 13:27:28.512~2019-09-23 13:42:30.682.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 13:42:24.682~2019-09-23 13:57:57.566.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 13:57:20.566~2019-09-23 14:12:59.139.txt',
]
# for i in attack_list:
#     label_h660[i]=1

for i in label_h660:
    for k in attack_list:
        f = k.split('~')[-1]
        if i.split('~')[-1] == k.split('~')[-1]:
            label_h660[i] = 1
            break
print(f'Attack Files:  {len(attack_list)} - {sum(label_h660.values())}')

Attack Files:  3 - 3


In [27]:
label_h660

{'/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 09:08:02.84~2019-09-23 09:23:45.704.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 09:23:45.704~2019-09-23 09:38:55.601.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 09:38:55.601~2019-09-23 09:54:07.697.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 09:54:07.697~2019-09-23 10:09:11.523.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 10:09:11.523~2019-09-23 10:24:12.318.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 10:24:12.318~2019-09-23 10:39:46.11.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 10:39:46.11~2019-09-23 10:54:51.309.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 10:54:51.309~2019-09-23 11:10:33.413.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 11:10:33.413~2019-09-23 11:25:43.15.txt': 0,
 '/home/shahidul2k9/data/optc/te

## h501

In [28]:
label_h501={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h501/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h501[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h501/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h501[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h501/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h501[test_path+f]=0

attack_list=[

 '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 10:15:28.241~2019-09-24 10:30:00.201.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 10:30:24.201~2019-09-24 10:45:02.7.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 10:45:20.7~2019-09-24 11:00:31.385.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 11:00:16.385~2019-09-24 11:16:09.755.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 11:15:12.755~2019-09-24 11:31:14.287.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 11:32:16.287~2019-09-24 11:46:31.541.txt',

 '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 13:04:00.804~2019-09-24 13:17:29.451.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 13:18:56.451~2019-09-24 13:32:46.454.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 13:33:52.454~2019-09-24 13:48:02.493.txt',

# adjust

#  'graph_9_24_h501/2019-09-24 11:47:12.541~2019-09-24 12:01:32.699.txt',
#  'graph_9_24_h501/2019-09-24 12:02:08.699~2019-09-24 12:16:34.904.txt',
#  'graph_9_24_h501/2019-09-24 12:32:00.112~2019-09-24 12:46:58.691.txt',
#  'graph_9_24_h501/2019-09-24 12:46:56.691~2019-09-24 13:02:02.804.txt',
#  'graph_9_24_h501/2019-09-24 16:22:24.281~2019-09-24 16:36:06.878.txt',

]
# for i in attack_list:
#     label_h501[i]=1

for i in label_h501:
    for k in attack_list:
        f = k.split('~')[-1]
        if i.split('~')[-1] == k.split('~')[-1]:
            label_h501[i] = 1
            break
print(f'Attack Files:  {len(attack_list)} - {sum(label_h501.values())}')

Attack Files:  9 - 9


In [29]:
label_h501

{'/home/shahidul2k9/data/optc/test/graph_9_23_h501/2019-09-23 09:06:52.727~2019-09-23 09:21:54.619.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h501/2019-09-23 09:21:54.619~2019-09-23 09:37:01.145.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h501/2019-09-23 09:37:01.145~2019-09-23 09:52:07.69.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h501/2019-09-23 09:52:07.69~2019-09-23 10:07:08.84.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h501/2019-09-23 10:07:08.84~2019-09-23 10:22:48.41.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h501/2019-09-23 10:22:48.41~2019-09-23 10:38:10.454.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h501/2019-09-23 10:38:10.454~2019-09-23 10:53:27.869.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h501/2019-09-23 10:53:27.869~2019-09-23 11:08:50.118.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h501/2019-09-23 11:08:50.118~2019-09-23 11:24:47.347.txt': 0,
 '/home/shahidul2k9/data/optc/test

## h051

In [30]:
label_h051={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h051/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h051[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h051/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h051[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h051/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h051[test_path+f]=0

attack_list=[
'/home/shahidul2k9/data/optc/test/graph_9_25_h051/2019-09-25 10:26:08.397~2019-09-25 10:41:40.247.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_25_h051/2019-09-25 10:41:04.247~2019-09-25 10:56:56.92.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_25_h051/2019-09-25 10:56:00.92~2019-09-25 11:12:03.608.txt',
]
# for i in attack_list:
#     label_h051[i]=1

for i in label_h051:
    for k in attack_list:
        f = k.split('~')[-1]
        if i.split('~')[-1] == k.split('~')[-1]:
            label_h051[i] = 1
            break
print(f'Attack Files:  {len(attack_list)} - {sum(label_h051.values())}')

Attack Files:  3 - 3


In [31]:
label_h051

{'/home/shahidul2k9/data/optc/test/graph_9_23_h051/2019-09-23 09:06:00.66~2019-09-23 09:21:07.896.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h051/2019-09-23 09:21:07.896~2019-09-23 09:36:24.501.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h051/2019-09-23 09:36:24.501~2019-09-23 09:51:30.238.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h051/2019-09-23 09:51:30.238~2019-09-23 10:06:30.739.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h051/2019-09-23 10:06:30.739~2019-09-23 10:21:35.62.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h051/2019-09-23 10:21:35.62~2019-09-23 10:36:37.274.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h051/2019-09-23 10:36:37.274~2019-09-23 10:51:57.908.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h051/2019-09-23 10:51:57.908~2019-09-23 11:07:00.719.txt': 0,
 '/home/shahidul2k9/data/optc/test/graph_9_23_h051/2019-09-23 11:07:00.719~2019-09-23 11:22:04.503.txt': 0,
 '/home/shahidul2k9/data/optc/t

## h207

In [36]:
label_h207={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h207/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h207[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h207/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h207[test_path+f]=0

test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h207/"

filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    label_h207[test_path+f]=0

attack_list=[

]
# for i in attack_list:
#     label_h207[i]=1

for i in label_h207:
    for k in attack_list:
        f = k.split('~')[-1]
        if i.split('~')[-1] == k.split('~')[-1]:
            label_h207[i] = 1
            break
print(f'Attack Files:  {len(attack_list)} - {sum(label_h207.values())}')

Attack Files:  0 - 0


# Anoamly detection

In [37]:
def classifier_evaluation(y_test, y_test_pred):
    # groundtruth, pred_value
    tn, fp, fn, tp =confusion_matrix(y_test, y_test_pred).ravel()
#     tn+=100
#     print(clf_name," : ")
    print('tn:',tn)
    print('fp:',fp)
    print('fn:',fn)
    print('tp:',tp)
    precision=tp/(tp+fp)
    recall = None
    if tp + fn == 0:
        recall = 1
    else:
        recall=tp/(tp+fn)
    accuracy=(tp+tn)/(tp+tn+fp+fn)
    fscore=2*(precision*recall)/(precision+recall)
    auc_val = None
    try:
        auc_val=roc_auc_score(y_test, y_test_pred)
    except Exception as e:
        print(f'Warn: failed to calculate acu_val {y_test} {y_test_pred}')
    print("precision:",precision)
    print("recall:",recall)
    print("fscore:",fscore)
    print("accuracy:",accuracy)
    print("auc_val:",auc_val)
    return precision,recall,fscore,accuracy,auc_val

## h201

### 9-23

In [38]:

node_IDF=torch.load("node_IDF_9_22_hosts")

# node_set_list=[]
history_list_9_23_h201=[]
tw_que=[]
his_tw={}
current_tw={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h201/"


file_l=os.listdir(test_path)
file_l.sort()
print(f'files {file_l}')
index_count=0
for f_path in (file_l):
    f=open(test_path+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']).split("_@")[-1],str(jdata['dstmsg']).split("_@")[-1]])
#         edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
        
#     df_list_9_22.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,test_path)
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_9_23_h201:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                print(f"{his_tw['name']=}")
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_9_23_h201.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

files ['2019-09-23 09:06:13.9~2019-09-23 09:21:19.885.txt', '2019-09-23 09:21:19.885~2019-09-23 09:36:28.932.txt', '2019-09-23 09:36:28.932~2019-09-23 09:51:40.187.txt', '2019-09-23 09:51:40.187~2019-09-23 10:06:52.508.txt', '2019-09-23 10:06:52.508~2019-09-23 10:21:54.256.txt', '2019-09-23 10:21:54.256~2019-09-23 10:37:17.566.txt', '2019-09-23 10:37:17.566~2019-09-23 10:52:41.476.txt', '2019-09-23 10:52:41.476~2019-09-23 11:08:03.86.txt', '2019-09-23 11:08:03.86~2019-09-23 11:23:05.136.txt', '2019-09-23 11:23:05.136~2019-09-23 11:38:30.698.txt', '2019-09-23 11:38:30.698~2019-09-23 11:53:39.57.txt', '2019-09-23 11:53:39.57~2019-09-23 12:08:55.34.txt', '2019-09-23 12:08:55.34~2019-09-23 12:24:01.487.txt', '2019-09-23 12:24:01.487~2019-09-23 12:39:14.95.txt', '2019-09-23 12:39:14.95~2019-09-23 12:54:14.286.txt', '2019-09-23 12:54:14.286~2019-09-23 13:09:50.95.txt', '2019-09-23 13:09:50.95~2019-09-23 13:24:56.43.txt', '2019-09-23 13:24:56.43~2019-09-23 13:40:05.109.txt', '2019-09-23 13:40

In [39]:
pred_label_h201={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h201/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h201[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h201/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h201[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h201/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h201[test_path+f]=0
    
    
    
    
for hl in history_list_9_23_h201:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>1000:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(*name_list, sep = "\n")
        for i in name_list:
            pred_label_h201["/home/shahidul2k9/data/optc/test/graph_9_23_h201/"+i]=1
        print(loss_count)

2019-09-23 09:06:13.9~2019-09-23 09:21:19.885.txt
2019-09-23 09:21:19.885~2019-09-23 09:36:28.932.txt
2019-09-23 09:36:28.932~2019-09-23 09:51:40.187.txt
2019-09-23 09:51:40.187~2019-09-23 10:06:52.508.txt
2019-09-23 10:06:52.508~2019-09-23 10:21:54.256.txt
2019-09-23 10:21:54.256~2019-09-23 10:37:17.566.txt
2019-09-23 10:37:17.566~2019-09-23 10:52:41.476.txt
2019-09-23 10:52:41.476~2019-09-23 11:08:03.86.txt
2019-09-23 11:08:03.86~2019-09-23 11:23:05.136.txt
2019-09-23 11:23:05.136~2019-09-23 11:38:30.698.txt
2019-09-23 11:38:30.698~2019-09-23 11:53:39.57.txt
2019-09-23 11:53:39.57~2019-09-23 12:08:55.34.txt
2019-09-23 12:08:55.34~2019-09-23 12:24:01.487.txt
2019-09-23 12:24:01.487~2019-09-23 12:39:14.95.txt
2019-09-23 12:39:14.95~2019-09-23 12:54:14.286.txt
2019-09-23 12:54:14.286~2019-09-23 13:09:50.95.txt
2019-09-23 13:09:50.95~2019-09-23 13:24:56.43.txt
2019-09-23 13:24:56.43~2019-09-23 13:40:05.109.txt
2019-09-23 13:40:05.109~2019-09-23 13:55:06.694.txt
2019-09-23 13:55:06.694~20

In [ ]:
# import os
# import re

# # Directory containing the files

# # Regular expression to match "tensor(x)"
# tensor_pattern = re.compile(r"tensor\((\d+)\)")
# for d in os.listdir("/home/shahidul2k9/data/optc/test"):
#     directory = "/home/shahidul2k9/data/optc/test" + "/" + d
#     # Iterate through files in the directory
#     for filename in os.listdir(directory):
        
#         print(filename)
#         # Replace "tensor(x)" with "x" in the filename
#         new_filename = re.sub(tensor_pattern, r"\1", filename)
        
#         # Rename the file if the name has changed
#         if new_filename != filename:
#             old_path = os.path.join(directory, filename)
#             new_path = os.path.join(directory, new_filename)
#             os.rename(old_path, new_path)
#             print(f'Renamed: "{filename}" -> "{new_filename}"')

2019-09-22 17:35:03.tensor(654)~2019-09-22 17:50:35.22.txt
Renamed: "2019-09-22 17:35:03.tensor(654)~2019-09-22 17:50:35.22.txt" -> "2019-09-22 17:35:03.654~2019-09-22 17:50:35.22.txt"
2019-09-22 16:18:09.tensor(932)~2019-09-22 16:33:32.947.txt
Renamed: "2019-09-22 16:18:09.tensor(932)~2019-09-22 16:33:32.947.txt" -> "2019-09-22 16:18:09.932~2019-09-22 16:33:32.947.txt"
2019-09-22 15:47:17.tensor(742)~2019-09-22 16:02:32.376.txt
Renamed: "2019-09-22 15:47:17.tensor(742)~2019-09-22 16:02:32.376.txt" -> "2019-09-22 15:47:17.742~2019-09-22 16:02:32.376.txt"
2019-09-22 04:35:31.tensor(906)~2019-09-22 04:50:40.909.txt
Renamed: "2019-09-22 04:35:31.tensor(906)~2019-09-22 04:50:40.909.txt" -> "2019-09-22 04:35:31.906~2019-09-22 04:50:40.909.txt"
2019-09-22 14:00:20.tensor(708)~2019-09-22 14:15:38.83.txt
Renamed: "2019-09-22 14:00:20.tensor(708)~2019-09-22 14:15:38.83.txt" -> "2019-09-22 14:00:20.708~2019-09-22 14:15:38.83.txt"
2019-09-22 16:48:45.tensor(41)~2019-09-22 17:03:58.893.txt
Renamed

In [40]:
# evaluate
y=[]
y_pred=[]
for i in label_h201:
    if i in pred_label_h201:
        y.append(label_h201[i])
        y_pred.append(pred_label_h201[i])
    else:
        print(f'Warn: missing file {i}')
    
classifier_evaluation(y,y_pred)

tn: 149
fp: 53
fn: 0
tp: 5
precision: 0.08620689655172414
recall: 1.0
fscore: 0.15873015873015872
accuracy: 0.7439613526570048
auc_val: 0.8688118811881188


(0.08620689655172414,
 1.0,
 0.15873015873015872,
 0.7439613526570048,
 0.8688118811881188)

### 9-24

In [41]:

node_IDF=torch.load("node_IDF_9_22_hosts")

# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h201/"


file_l=os.listdir(test_path)
file_l.sort()
index_count=0
for f_path in (file_l):
    f=open(test_path+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']).split("_@")[-1],str(jdata['dstmsg']).split("_@")[-1]])
#         edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
        
#     df_list_9_22.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,test_path)
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                print(f"{his_tw['name']=}")
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

index_count: 0
thr: 4.021752052795872
2019-09-24 00:00:00.17~2019-09-24 00:15:56.169.txt    7.548702148044056  count: 1115  percentage: 0.02532249273255814  node count: 357  edge count: 366
index_count: 1
thr: 2.479389746227952
node: gpupdate.exe  IDF: 6.150602768446279
node: System  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml.new  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/lwabeat//lwabeat.exe  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/$Extend/$UsnJrnl:$J  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
his_tw['name']='2019-09-24 00:00:00.17~2019-09-24 00:15:56.169.txt'
2019-09-24 00:15:56.169~2019-09-24 00:31:22.853.txt    3.9867055126849342  count: 544  percentage: 0.020432692307692308  node count: 170  edge count: 202
index_count: 2
thr: 2.2662615968510713
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
node: /Device/Harddi

In [42]:

for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>10000:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(name_list)
#         for i in name_list:
#             pred_label[i]=1
        print(loss_count)

['2019-09-24 00:00:00.17~2019-09-24 00:15:56.169.txt', '2019-09-24 00:15:56.169~2019-09-24 00:31:22.853.txt', '2019-09-24 00:31:22.853~2019-09-24 00:46:30.673.txt', '2019-09-24 00:46:30.673~2019-09-24 01:01:58.802.txt', '2019-09-24 01:01:58.802~2019-09-24 01:17:02.278.txt', '2019-09-24 01:17:02.278~2019-09-24 01:32:18.817.txt', '2019-09-24 01:32:18.817~2019-09-24 01:47:20.516.txt', '2019-09-24 01:47:20.516~2019-09-24 02:02:29.364.txt', '2019-09-24 02:02:29.364~2019-09-24 02:17:40.714.txt', '2019-09-24 02:17:40.714~2019-09-24 02:32:50.238.txt', '2019-09-24 02:32:50.238~2019-09-24 02:48:15.298.txt', '2019-09-24 02:48:15.298~2019-09-24 03:03:48.872.txt', '2019-09-24 03:03:48.872~2019-09-24 03:19:07.59.txt', '2019-09-24 03:19:07.59~2019-09-24 03:34:51.361.txt', '2019-09-24 03:34:51.361~2019-09-24 03:49:53.922.txt', '2019-09-24 03:49:53.922~2019-09-24 04:05:01.182.txt', '2019-09-24 04:05:01.182~2019-09-24 04:20:16.275.txt', '2019-09-24 04:20:16.275~2019-09-24 04:35:25.138.txt', '2019-09-24 

### 9-25

In [43]:

node_IDF=torch.load("node_IDF_9_22_hosts")

# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h201/"


file_l=os.listdir(test_path)
file_l.sort()
index_count=0
for f_path in (file_l):
    f=open(test_path+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']).split("_@")[-1],str(jdata['dstmsg']).split("_@")[-1]])
#         edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,test_path)
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                print(f"{his_tw['name']=}")
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
  
    index_count+=1

    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))


index_count: 0
thr: 3.8698245624760657
2019-09-25 00:00:00.4~2019-09-25 00:15:35.902.txt    7.599501787071467  count: 1119  percentage: 0.023250498670212765  node count: 516  edge count: 525
index_count: 1
thr: 2.1544562676689414
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml.new  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/$Extend/$UsnJrnl:$J  IDF: 6.150602768446279
node: System  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/lwabeat//lwabeat.exe  IDF: 6.150602768446279
his_tw['name']='2019-09-25 00:00:00.4~2019-09-25 00:15:35.902.txt'
2019-09-25 00:15:35.902~2019-09-25 00:30:53.501.txt    3.1713104798243594  count: 377  percentage: 0.018408203125  node count: 61  edge count: 69
index_count: 2
thr: 2.189719087907219
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml.new  IDF:

In [44]:

# pred_label={}

    
# filelist = os.listdir("graph_9_23")
# for f in filelist:
#     pred_label[f]=0


for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>10000:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(name_list)
#         for i in name_list:
#             pred_label[i]=1
        print(loss_count)

['2019-09-25 00:00:00.4~2019-09-25 00:15:35.902.txt', '2019-09-25 00:15:35.902~2019-09-25 00:30:53.501.txt', '2019-09-25 00:30:53.501~2019-09-25 00:45:55.406.txt', '2019-09-25 00:45:55.406~2019-09-25 01:00:56.815.txt', '2019-09-25 01:00:56.815~2019-09-25 01:16:00.888.txt', '2019-09-25 01:16:00.888~2019-09-25 01:31:09.172.txt', '2019-09-25 01:31:09.172~2019-09-25 01:46:12.38.txt', '2019-09-25 01:46:12.38~2019-09-25 02:01:23.19.txt', '2019-09-25 02:01:23.19~2019-09-25 02:16:33.682.txt', '2019-09-25 02:16:33.682~2019-09-25 02:32:33.877.txt', '2019-09-25 02:32:33.877~2019-09-25 02:47:57.447.txt', '2019-09-25 02:47:57.447~2019-09-25 03:03:02.798.txt', '2019-09-25 03:03:02.798~2019-09-25 03:18:23.568.txt', '2019-09-25 03:18:23.568~2019-09-25 03:33:28.273.txt', '2019-09-25 03:33:28.273~2019-09-25 03:48:56.513.txt', '2019-09-25 03:48:56.513~2019-09-25 04:04:07.678.txt', '2019-09-25 04:04:07.678~2019-09-25 04:19:41.227.txt', '2019-09-25 04:19:41.227~2019-09-25 04:35:00.256.txt', '2019-09-25 04:

## h402

In [45]:

node_IDF=torch.load("node_IDF_9_22_hosts")

# node_set_list=[]
history_list_9_23_h402=[]
tw_que=[]
his_tw={}
current_tw={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h402/"


file_l=os.listdir(test_path)
file_l.sort()
index_count=0
for f_path in (file_l):
    f=open(test_path+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']).split("_@")[-1],str(jdata['dstmsg']).split("_@")[-1]])
#         edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
        
#     df_list_9_22.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,test_path)
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_9_23_h402:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                print(f"{his_tw['name']=}")
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_9_23_h402.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

index_count: 0
thr: 5.1812313303463435
2019-09-23 09:06:18.504~2019-09-23 09:21:19.667.txt    7.433533264503319  count: 2390  percentage: 0.036468505859375  node count: 1368  edge count: 1381
index_count: 1
thr: 3.784446106827718
node: reg.exe  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
node: %SystemRoot%/system32/csrss.exe  IDF: 6.150602768446279
node: taskkill.exe  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/$Extend/$UsnJrnl:$J  IDF: 6.150602768446279
node: taskhostw.exe  IDF: 6.150602768446279
node: geckodriver.exe  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData  IDF: 6.150602768446279
node: PING.EXE  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/$Mft  IDF: 6.150602768446279
node: System  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/Microsoft/Network/Downloader/qmgr.db  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/lwabeat//lwabeat.exe  IDF: 6.15060276844627

In [46]:
pred_label_h402={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h402/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h402[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h402/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h402[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h402/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h402[test_path+f]=0
    
    
    
    
for hl in history_list_9_23_h402:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>1000:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(*name_list, sep = "\n")
        for i in name_list:
            pred_label_h402["/home/shahidul2k9/data/optc/test/graph_9_23_h402/"+i]=1
        print(loss_count)

2019-09-23 09:06:18.504~2019-09-23 09:21:19.667.txt
2019-09-23 09:21:19.667~2019-09-23 09:37:02.486.txt
2019-09-23 09:37:02.486~2019-09-23 09:52:09.344.txt
2019-09-23 09:52:09.344~2019-09-23 10:07:16.51.txt
2019-09-23 10:07:16.51~2019-09-23 10:22:30.173.txt
2019-09-23 10:22:30.173~2019-09-23 10:37:47.93.txt
2019-09-23 10:37:47.93~2019-09-23 10:53:18.749.txt
2019-09-23 10:53:18.749~2019-09-23 11:08:44.611.txt
2019-09-23 11:08:44.611~2019-09-23 11:24:02.2.txt
2019-09-23 11:24:02.2~2019-09-23 11:39:03.722.txt
2019-09-23 11:39:03.722~2019-09-23 11:54:19.929.txt
2019-09-23 11:54:19.929~2019-09-23 12:09:35.596.txt
2019-09-23 12:09:35.596~2019-09-23 12:24:47.182.txt
2019-09-23 12:24:47.182~2019-09-23 12:40:00.761.txt
2019-09-23 12:40:00.761~2019-09-23 12:55:03.39.txt
2019-09-23 12:55:03.39~2019-09-23 13:10:06.429.txt
2019-09-23 13:10:06.429~2019-09-23 13:25:09.374.txt
2019-09-23 13:25:09.374~2019-09-23 13:40:21.268.txt
2019-09-23 13:40:21.268~2019-09-23 13:55:31.31.txt
2019-09-23 13:55:31.31~

In [47]:
# Evalaute
y=[]
y_pred=[]
for i in label_h402:
    if i in pred_label_h402:
        y.append(label_h402[i])
        y_pred.append(pred_label_h402[i])
    else:
        print(f'Warn: missing file {i}')
    
classifier_evaluation(y,y_pred)

tn: 151
fp: 56
fn: 0
tp: 2
precision: 0.034482758620689655
recall: 1.0
fscore: 0.06666666666666667
accuracy: 0.7320574162679426
auc_val: 0.8647342995169083


(0.034482758620689655,
 1.0,
 0.06666666666666667,
 0.7320574162679426,
 0.8647342995169083)

## h660

In [48]:

node_IDF=torch.load("node_IDF_9_22_hosts")

# node_set_list=[]
history_list_9_23_h660=[]
tw_que=[]
his_tw={}
current_tw={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h660/"


file_l=os.listdir(test_path)
file_l.sort()
index_count=0
for f_path in (file_l):
    f=open(test_path+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']).split("_@")[-1],str(jdata['dstmsg']).split("_@")[-1]])
#         edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
        
#     df_list_9_22.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,test_path)
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_9_23_h660:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                print(f"{his_tw['name']=}")
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_9_23_h660.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

index_count: 0
thr: 4.758069320156217
2019-09-23 09:08:02.84~2019-09-23 09:23:45.704.txt    7.521229893512573  count: 1371  percentage: 0.03265529725609756  node count: 697  edge count: 706
index_count: 1
thr: 2.3448855340842507
node: /Device/HarddiskVolume1/ProgramData/Microsoft/Diagnosis/DownloadedSettings/utc.app.json.new  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
node: schtasks.exe  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/$Extend/$UsnJrnl:$J  IDF: 6.150602768446279
node: PING.EXE  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/Microsoft/Diagnosis/DownloadedSettings/utc.cert.json.new  IDF: 6.150602768446279
node: System  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/lwabeat//lwabeat.exe  IDF: 6.150602768446279
his_tw['name']='2019-09-23 09:08:02.84~2019-09-23 09:23:45.704.txt'
2019-09-23 09:23:45.704~2019-09-23 09:38:55.601.txt    3.307115592142017  count: 843  percentage: 0.0235

In [49]:
pred_label_h660={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h660/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h660[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h660/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h660[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h660/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h660[test_path+f]=0
    
    
    
    
for hl in history_list_9_23_h660:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(*name_list, sep = "\n")
        for i in name_list:
            pred_label_h660["/home/shahidul2k9/data/optc/test/graph_9_23_h660/"+i]=1
        print(loss_count)
        
        
# evalute
y=[]
y_pred=[]
for i in label_h660:
    if i in pred_label_h660:
        y.append(label_h660[i])
        y_pred.append(pred_label_h660[i])
    else:
        print(f'Warn file not found {i}')
    
classifier_evaluation(y,y_pred)

2019-09-23 09:08:02.84~2019-09-23 09:23:45.704.txt
2019-09-23 09:23:45.704~2019-09-23 09:38:55.601.txt
2019-09-23 09:38:55.601~2019-09-23 09:54:07.697.txt
2019-09-23 09:54:07.697~2019-09-23 10:09:11.523.txt
2019-09-23 10:09:11.523~2019-09-23 10:24:12.318.txt
2019-09-23 10:24:12.318~2019-09-23 10:39:46.11.txt
2019-09-23 10:39:46.11~2019-09-23 10:54:51.309.txt
2019-09-23 10:54:51.309~2019-09-23 11:10:33.413.txt
2019-09-23 11:10:33.413~2019-09-23 11:25:43.15.txt
2019-09-23 11:25:43.15~2019-09-23 11:40:55.982.txt
2019-09-23 11:40:55.982~2019-09-23 11:56:01.799.txt
2019-09-23 11:56:01.799~2019-09-23 12:11:12.95.txt
2019-09-23 12:11:12.95~2019-09-23 12:26:51.222.txt
2019-09-23 12:26:51.222~2019-09-23 12:42:01.823.txt
2019-09-23 12:42:01.823~2019-09-23 12:57:16.878.txt
2019-09-23 12:57:16.878~2019-09-23 13:12:20.85.txt
2019-09-23 13:12:20.85~2019-09-23 13:27:27.512.txt
2019-09-23 13:27:27.512~2019-09-23 13:42:30.682.txt
2019-09-23 13:42:30.682~2019-09-23 13:57:57.566.txt
2019-09-23 13:57:57.5

(0.05172413793103448,
 1.0,
 0.09836065573770492,
 0.7355769230769231,
 0.8658536585365854)

## h501

In [50]:

node_IDF=torch.load("node_IDF_9_22_hosts")

# node_set_list=[]
history_list_9_24_h501=[]
tw_que=[]
his_tw={}
current_tw={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h501/"


file_l=os.listdir(test_path)
file_l.sort()
index_count=0
for f_path in (file_l):
    f=open(test_path+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']).split("_@")[-1],str(jdata['dstmsg']).split("_@")[-1]])
#         edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
        
#     df_list_9_22.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,test_path)
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_9_24_h501:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                print(f"{his_tw['name']=}")
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_9_24_h501.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

index_count: 0
thr: 3.9324595994668825
2019-09-24 00:00:00.17~2019-09-24 00:15:03.448.txt    7.418346304068746  count: 1006  percentage: 0.022847020348837208  node count: 315  edge count: 330
index_count: 1
thr: 2.324357758812844
node: System  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/$LogFile  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/$Extend/$UsnJrnl:$J  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/lwabeat//lwabeat.exe  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
his_tw['name']='2019-09-24 00:00:00.17~2019-09-24 00:15:03.448.txt'
2019-09-24 00:15:03.448~2019-09-24 00:30:57.378.txt    3.376249861237772  count: 547  percentage: 0.0213671875  node count: 72  edge count: 91
index_count: 2
thr: 2.5159224604970825
node: /Device/HarddiskVolume1/$LogFile  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/Progra

In [51]:
pred_label_h501={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h501/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h501[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h501/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h501[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h501/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h501[test_path+f]=0
    
    
    
    
for hl in history_list_9_24_h501:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>200:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(*name_list, sep = "\n")
        for i in name_list:
            pred_label_h501["/home/shahidul2k9/data/optc/test/graph_9_24_h501/"+i]=1
        print(loss_count)
        
        
# evaluate
y=[]
y_pred=[]
for i in label_h501:
    if i in pred_label_h501:
        y.append(label_h501[i])
        y_pred.append(pred_label_h501[i])
    else:
        print(f'Warn file not found {f}')
    
classifier_evaluation(y,y_pred)

2019-09-24 00:00:00.17~2019-09-24 00:15:03.448.txt
2019-09-24 00:15:03.448~2019-09-24 00:30:57.378.txt
2019-09-24 00:30:57.378~2019-09-24 00:46:01.19.txt
2019-09-24 00:46:01.19~2019-09-24 01:01:18.716.txt
2019-09-24 01:01:18.716~2019-09-24 01:16:34.3.txt
2019-09-24 01:16:34.3~2019-09-24 01:31:41.697.txt
2019-09-24 01:31:41.697~2019-09-24 01:46:49.856.txt
2019-09-24 01:46:49.856~2019-09-24 02:02:32.588.txt
2019-09-24 02:02:32.588~2019-09-24 02:17:45.294.txt
2019-09-24 02:17:45.294~2019-09-24 02:33:01.295.txt
2019-09-24 02:33:01.295~2019-09-24 02:48:28.246.txt
2019-09-24 02:48:28.246~2019-09-24 03:04:02.969.txt
2019-09-24 03:04:02.969~2019-09-24 03:19:37.529.txt
2019-09-24 03:19:37.529~2019-09-24 03:35:02.235.txt
2019-09-24 03:35:02.235~2019-09-24 03:50:11.934.txt
2019-09-24 03:50:11.934~2019-09-24 04:05:24.7.txt
2019-09-24 04:05:24.7~2019-09-24 04:20:34.972.txt
2019-09-24 04:20:34.972~2019-09-24 04:35:48.303.txt
2019-09-24 04:35:48.303~2019-09-24 04:51:16.912.txt
2019-09-24 04:51:16.912

(0.09574468085106383, 1.0, 0.17475728155339806, 0.5933014354066986, 0.7875)

## h051

In [52]:
# node_IDF=torch.load("node_IDF_4_6")
node_IDF=torch.load("node_IDF_9_22_hosts")

# node_set_list=[]
history_list_9_25_h051=[]
tw_que=[]
his_tw={}
current_tw={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h051/"


file_l=os.listdir(test_path)
file_l.sort()
index_count=0
for f_path in (file_l):
    f=open(test_path+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']).split("_@")[-1],str(jdata['dstmsg']).split("_@")[-1]])
#         edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
        
#     df_list_9_22.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,test_path)
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_9_25_h051:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                print(f"{his_tw['name']=}")
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_9_25_h051.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

index_count: 0
thr: 4.114306066759045
2019-09-25 00:00:00.76~2019-09-25 00:15:08.283.txt    7.568265404125485  count: 1292  percentage: 0.026845079787234043  node count: 611  edge count: 626
index_count: 1
thr: 2.9529085070201684
node: /Device/HarddiskVolume1/ProgramData/Microsoft/Diagnosis/DownloadedSettings/utc.app.json.new  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/$Extend/$UsnJrnl:$J  IDF: 6.150602768446279
node: schtasks.exe  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/Microsoft/Diagnosis/DownloadedSettings/utc.cert.json.new  IDF: 6.150602768446279
node: System  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/lwabeat//lwabeat.exe  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml.new  IDF: 6.150602768446279
his_tw['name']='2019-09-25 00:00:00.76~2019-09-25 00:15:08.283.txt'
2019-09-25 00:15:08.283~2019-09-25 00:30:59.473.txt 

In [53]:
pred_label_h051={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h051/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h051[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h051/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h051[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h051/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h051[test_path+f]=0
    
    
    
    
for hl in history_list_9_25_h051:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>2000:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])

        print(*name_list, sep = "\n")
        for i in name_list:
            pred_label_h051["/home/shahidul2k9/data/optc/test/graph_9_25_h051/"+i]=1
        print(loss_count)
        
        
# evaluate
y=[]
y_pred=[]
for i in label_h051:
    y.append(label_h051[i])
    y_pred.append(pred_label_h051[i])
    
classifier_evaluation(y,y_pred)

2019-09-25 00:00:00.76~2019-09-25 00:15:08.283.txt
2019-09-25 00:15:08.283~2019-09-25 00:30:59.473.txt
2019-09-25 00:30:59.473~2019-09-25 00:46:04.697.txt
2019-09-25 00:46:04.697~2019-09-25 01:01:19.572.txt
2019-09-25 01:01:19.572~2019-09-25 01:16:39.846.txt
2019-09-25 01:16:39.846~2019-09-25 01:31:45.444.txt
2019-09-25 01:31:45.444~2019-09-25 01:46:58.862.txt
2019-09-25 01:46:58.862~2019-09-25 02:02:01.998.txt
2019-09-25 02:02:01.998~2019-09-25 02:17:35.906.txt
2019-09-25 02:17:35.906~2019-09-25 02:32:39.45.txt
2019-09-25 02:32:39.45~2019-09-25 02:47:42.8.txt
2019-09-25 02:47:42.8~2019-09-25 03:02:54.764.txt
2019-09-25 03:02:54.764~2019-09-25 03:18:31.924.txt
2019-09-25 03:18:31.924~2019-09-25 03:34:01.95.txt
2019-09-25 03:34:01.95~2019-09-25 03:49:08.672.txt
2019-09-25 03:49:08.672~2019-09-25 04:04:38.557.txt
2019-09-25 04:04:38.557~2019-09-25 04:20:05.719.txt
2019-09-25 04:20:05.719~2019-09-25 04:35:36.83.txt
2019-09-25 04:35:36.83~2019-09-25 04:51:03.881.txt
2019-09-25 04:51:03.881

(0.05263157894736842, 1.0, 0.1, 0.7403846153846154, 0.8682926829268293)

## h207

In [54]:

node_IDF=torch.load("node_IDF_9_22_hosts")

# node_set_list=[]
history_list_9_25_h207=[]
tw_que=[]
his_tw={}
current_tw={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h207/"


file_l=os.listdir(test_path)
file_l.sort()
index_count=0
for f_path in (file_l):
    f=open(test_path+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']).split("_@")[-1],str(jdata['dstmsg']).split("_@")[-1]])
#         edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
        
#     df_list_9_22.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,test_path)
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_9_25_h207:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                print(f"{his_tw['name']=}")
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_9_25_h207.append(temp_hq)
  
    index_count+=1

    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

index_count: 0
thr: 3.9483694102521554
2019-09-25 00:00:00.2~2019-09-25 00:15:02.181.txt    7.566797969129254  count: 1102  percentage: 0.023395040760869564  node count: 432  edge count: 440
index_count: 1
thr: 2.315308984218874
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml.new  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/$Extend/$UsnJrnl:$J  IDF: 6.150602768446279
node: System  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/lwabeat//lwabeat.exe  IDF: 6.150602768446279
his_tw['name']='2019-09-25 00:00:00.2~2019-09-25 00:15:02.181.txt'
2019-09-25 00:15:02.181~2019-09-25 00:30:14.189.txt    3.3676748587849827  count: 466  percentage: 0.02275390625  node count: 54  edge count: 60
index_count: 2
thr: 2.370891662364302
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml  IDF: 6.150602768446279
node: /Device/HarddiskVolume1/ProgramData/lwa/.winlogbeat.yml.new  IDF: 6

In [55]:
pred_label_h207={}

test_path="/home/shahidul2k9/data/optc/test/graph_9_23_h207/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h207[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_24_h207/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h207[test_path+f]=0
    
test_path="/home/shahidul2k9/data/optc/test/graph_9_25_h207/"
    
filelist = os.listdir(test_path)
filelist.sort()
for f in filelist:
    pred_label_h207[test_path+f]=0
    
    
    
    
for hl in history_list_9_25_h207:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>2000:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])

        print(*name_list, sep = "\n")
        for i in name_list:
            pred_label_h207["/home/shahidul2k9/data/optc/test/graph_9_25_h207/"+i]=1
        print(loss_count)
        
        


2019-09-25 00:00:00.2~2019-09-25 00:15:02.181.txt
2019-09-25 00:15:02.181~2019-09-25 00:30:14.189.txt
2019-09-25 00:30:14.189~2019-09-25 00:45:17.983.txt
2019-09-25 00:45:17.983~2019-09-25 01:00:23.44.txt
2019-09-25 01:00:23.44~2019-09-25 01:15:31.611.txt
2019-09-25 01:15:31.611~2019-09-25 01:31:13.354.txt
2019-09-25 01:31:13.354~2019-09-25 01:46:31.134.txt
2019-09-25 01:46:31.134~2019-09-25 02:01:54.657.txt
2019-09-25 02:01:54.657~2019-09-25 02:17:35.923.txt
2019-09-25 02:17:35.923~2019-09-25 02:32:48.475.txt
2019-09-25 02:32:48.475~2019-09-25 02:48:07.82.txt
2019-09-25 02:48:07.82~2019-09-25 03:03:57.166.txt
2019-09-25 03:03:57.166~2019-09-25 03:20:04.778.txt
2019-09-25 03:20:04.778~2019-09-25 03:35:08.118.txt
2019-09-25 03:35:08.118~2019-09-25 03:50:39.313.txt
2019-09-25 03:50:39.313~2019-09-25 04:05:53.677.txt
2019-09-25 04:05:53.677~2019-09-25 04:21:01.308.txt
2019-09-25 04:21:01.308~2019-09-25 04:36:33.725.txt
2019-09-25 04:36:33.725~2019-09-25 04:51:51.193.txt
2019-09-25 04:51:5

In [56]:
# evaluate
y=[]
y_pred=[]
for i in label_h207:
    y.append(label_h207[i])
    y_pred.append(pred_label_h207[i])
    
    
classifier_evaluation(y,y_pred)

tn: 151
fp: 56
fn: 0
tp: 0
Warn: failed to calculate acu_val [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

(0.0, 1, 0.0, 0.7294685990338164, None)

## Overall evaluation

In [69]:


y=[]
y_pred=[]
#TODO: Investigate why missing in preprocessing steps
for i in label_h201:
    y.append(label_h201[i])
    y_pred.append(pred_label_h201[i])

for i in label_h402:
    y.append(label_h402[i])
    y_pred.append(pred_label_h402[i])

for i in label_h660:
    y.append(label_h660[i])
    y_pred.append(pred_label_h660[i])

for i in label_h501:
    y.append(label_h501[i])
    y_pred.append(pred_label_h501[i])

for i in label_h051:
    y.append(label_h051[i])
    y_pred.append(pred_label_h051[i])

for i in label_h207:
    y.append(label_h207[i])
    y_pred.append(pred_label_h207[i])
    
    
classifier_evaluation(y,y_pred)

tn: 867
fp: 359
fn: 0
tp: 22
precision: 0.05774278215223097
recall: 1.0
fscore: 0.10918114143920596
accuracy: 0.7123397435897436
auc_val: 0.8535889070146819


(0.05774278215223097,
 1.0,
 0.10918114143920596,
 0.7123397435897436,
 0.8535889070146819)

# Attack investigation

In [60]:
replace_dic={
    '.pyc':'*.pyc',
#     '.dll':'*.dll',
#     '.DLL':'*.DLL',
}

def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    if '->' in path_name:
        if 'outbound' in path_name:
            msg=re.findall("->(.*?):",path_name)[0]
            return msg
        elif 'inbound' in path_name:
            msg=re.findall("#(.*?):",path_name)[0]
            return msg
    return path_name

## Count the number of attack edges

In [59]:
label_df=pd.read_csv("./labels.csv")

FileNotFoundError: [Errno 2] No such file or directory: './labels.csv'

### h201

In [61]:
nodes_attack_h201={}
attack_node_set_h201=set()
edges_attack_hashset_h201=set()
h201_df_label=label_df[label_df['hostname']=='SysClient0201.systemia.com']

for idx,row in h201_df_label.iterrows():
    
    srcflag=False
    dstflag=False
    if row['objectID'] in node_uuid2path:
        nodes_attack_h201[row['objectID']]=replace_path_name(node_uuid2path[row['objectID']])

        dstflag=True
    if row['actorID'] in node_uuid2path:
        nodes_attack_h201[row['actorID']]=replace_path_name(node_uuid2path[row['actorID']])
        
        srcflag=True
    if srcflag and dstflag and row['action'] in rel2id:    
        temp_edge=replace_path_name(node_uuid2path[row['actorID']])+','+replace_path_name(node_uuid2path[row['objectID']])+','+str(datetime_to_timestamp_US(row['timestamp']))
        edges_attack_hashset_h201.add(hashgen(temp_edge))

for i in nodes_attack_h201:
    attack_node_set_h201.add(nodes_attack_h201[i])

NameError: name 'label_df' is not defined

### h402

In [62]:
nodes_attack_h402={}
attack_node_set_h402=set()
edges_attack_hashset_h402=set()
h402_df_label=label_df[label_df['hostname']=='SysClient0402.systemia.com']

for idx,row in h402_df_label.iterrows():
    
    srcflag=False
    dstflag=False
    if row['objectID'] in node_uuid2path:
        nodes_attack_h402[row['objectID']]=replace_path_name(node_uuid2path[row['objectID']])
        dstflag=True
    if row['actorID'] in node_uuid2path:
        nodes_attack_h402[row['actorID']]=replace_path_name(node_uuid2path[row['actorID']])
        
        srcflag=True
    if srcflag and dstflag and row['action'] in rel2id:    
        temp_edge=replace_path_name(node_uuid2path[row['actorID']])+','+replace_path_name(node_uuid2path[row['objectID']])+','+str(datetime_to_timestamp_US(row['timestamp']))
        edges_attack_hashset_h402.add(hashgen(temp_edge))

for i in nodes_attack_h402:
    attack_node_set_h402.add(nodes_attack_h402[i])

NameError: name 'label_df' is not defined

In [63]:
len(edges_attack_hashset_h402)

0

### h660

In [64]:
nodes_attack_h660={}
attack_node_set_h660=set()
edges_attack_hashset_h660=set()
h660_df_label=label_df[label_df['hostname']=='SysClient0660.systemia.com']

for idx,row in h660_df_label.iterrows():
    
    srcflag=False
    dstflag=False
    if row['objectID'] in node_uuid2path:
        nodes_attack_h660[row['objectID']]=replace_path_name(node_uuid2path[row['objectID']])
        dstflag=True
    if row['actorID'] in node_uuid2path:
        nodes_attack_h660[row['actorID']]=replace_path_name(node_uuid2path[row['actorID']])
        srcflag=True
    if srcflag and dstflag and row['action'] in rel2id:    
        temp_edge=replace_path_name(node_uuid2path[row['actorID']])+','+replace_path_name(node_uuid2path[row['objectID']])+','+str(datetime_to_timestamp_US(row['timestamp']))
        edges_attack_hashset_h660.add(hashgen(temp_edge))

for i in nodes_attack_h660:
    attack_node_set_h660.add(nodes_attack_h660[i])

NameError: name 'label_df' is not defined

In [ ]:
len(edges_attack_hashset_h660)

### h501

In [ ]:
nodes_attack_h501={}
attack_node_set_h501=set()
edges_attack_hashset_h501=set()
h501_df_label=label_df[label_df['hostname']=='SysClient0501.systemia.com']

for idx,row in h501_df_label.iterrows():
    
    srcflag=False
    dstflag=False
    if row['objectID'] in node_uuid2path:
        nodes_attack_h501[row['objectID']]=replace_path_name(node_uuid2path[row['objectID']])
        dstflag=True
    if row['actorID'] in node_uuid2path:
        nodes_attack_h501[row['actorID']]=replace_path_name(node_uuid2path[row['actorID']])
        srcflag=True
    if srcflag and dstflag and row['action'] in rel2id:    
        temp_edge=replace_path_name(node_uuid2path[row['actorID']])+','+replace_path_name(node_uuid2path[row['objectID']])+','+str(datetime_to_timestamp_US(row['timestamp']))
        edges_attack_hashset_h501.add(hashgen(temp_edge))

for i in nodes_attack_h501:
    attack_node_set_h501.add(nodes_attack_h501[i])

### h051

In [ ]:
nodes_attack_h051={}
attack_node_set_h051=set()
edges_attack_hashset_h051=set()
h051_df_label=label_df[label_df['hostname']=='SysClient0051.systemia.com']

for idx,row in h051_df_label.iterrows():
    
    srcflag=False
    dstflag=False
    if row['objectID'] in node_uuid2path:
        nodes_attack_h051[row['objectID']]=replace_path_name(node_uuid2path[row['objectID']])
        dstflag=True
    if row['actorID'] in node_uuid2path:
        nodes_attack_h051[row['actorID']]=replace_path_name(node_uuid2path[row['actorID']])
        srcflag=True
    if srcflag and dstflag and row['action'] in rel2id:    
        temp_edge=replace_path_name(node_uuid2path[row['actorID']])+','+replace_path_name(node_uuid2path[row['objectID']])+','+str(datetime_to_timestamp_US(row['timestamp']))
        edges_attack_hashset_h051.add(hashgen(temp_edge))

for i in nodes_attack_h051:
    attack_node_set_h051.add(nodes_attack_h051[i])

## h201

In [65]:
attack_list=[
    '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 09:52:00.187~2019-09-23 10:06:52.508.txt',
  '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 11:23:44.136~2019-09-23 11:38:30.698.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 11:38:40.698~2019-09-23 11:53:39.57.txt',
 
 '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 12:38:24.95~2019-09-23 12:54:14.286.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 12:55:28.286~2019-09-23 13:09:50.95.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 13:10:24.95~2019-09-23 13:24:56.43.txt',
    '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 15:24:48.426~2019-09-23 15:42:02.967.txt',
            ]


In [66]:
from datetime import datetime, timezone
attack_dic=[]
for a in attack_list:
    temp_dic={}
    aa=a.split("/")[-1]
    sp=aa.split('~')
    start_timestamp=sp[0].split('.')[0]
    end_timestamp=sp[1].split('.')[0]
    temp_dic['start']=datetime_to_timestamp_US(start_timestamp)
    temp_dic['end']=datetime_to_timestamp_US(end_timestamp)
    temp_dic['file']=a
    attack_dic.append(temp_dic)
    print(temp_dic)

{'start': 1569246720000, 'end': 1569247612000, 'file': '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 09:52:00.187~2019-09-23 10:06:52.508.txt'}
{'start': 1569252224000, 'end': 1569253110000, 'file': '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 11:23:44.136~2019-09-23 11:38:30.698.txt'}
{'start': 1569253120000, 'end': 1569254019000, 'file': '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 11:38:40.698~2019-09-23 11:53:39.57.txt'}
{'start': 1569256704000, 'end': 1569257654000, 'file': '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 12:38:24.95~2019-09-23 12:54:14.286.txt'}
{'start': 1569257728000, 'end': 1569258590000, 'file': '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 12:55:28.286~2019-09-23 13:09:50.95.txt'}
{'start': 1569258624000, 'end': 1569259496000, 'file': '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 13:10:24.95~2019-09-23 13:24:56.43.txt'}
{'start': 1569266688000, 'end': 1569267722000, 'fil

In [67]:

original_edges_count=0
hash2msg={}
graphs=[]
gg=nx.DiGraph()
count=0
# file_list=os.listdir("./test_day_data4_10_emb100/")
for path in tqdm(attack_list):
#     print(path)
    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            count+=1
            l=line.strip()
            jdata=eval(l)
#             temp_key=jdata['srcmsg']+jdata['dstmsg']+jdata['edge_type']
#             if temp_key in train_edge_set:
#                 jdata['loss']=(jdata['loss']-train_edge_set[temp_key]) if jdata['loss']>=train_edge_set[temp_key] else 0  
#             jdata['loss']=abs(jdata['loss']-train_edge_set[temp_key])  if temp_key in train_edge_set else jdata['loss']
            edge_list.append(jdata)
            
        edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True) 
        original_edges_count+=len(edge_list)
        
        loss_list=[]
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean=mean(loss_list)
        loss_std=std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr=loss_mean+1.5*loss_std
#         thr=-99
        print("thr:",thr)
        for e in edge_list:
            if e['loss']>thr:    
#             if True:  
#                 if "'/home/admin/profile'" in e['srcmsg'] or " '/home/admin/profile'" in e['dstmsg']:
#                     print(e['srcmsg'])
#                     print(e['dstmsg'])
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))),loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
                
                hash2msg[str(hashgen(replace_path_name(e['srcmsg'])))]=replace_path_name(e['srcmsg'])
                hash2msg[str(hashgen(replace_path_name(e['dstmsg'])))]=replace_path_name(e['dstmsg'])

#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
        print(path)
        print("tempg edges:",len(tempg.edges))
        print("tempg nodes:",len(tempg.nodes))
        print("tempg weakly components:",nx.number_weakly_connected_components(tempg))
        
        print("gg edges:",len(gg.edges))
        print("gg nodes:",len(gg.nodes))
        print("gg weakly components:",nx.number_weakly_connected_components(gg))
        print(f"{original_edges_count=}")
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

  0%|          | 0/7 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 09:52:00.187~2019-09-23 10:06:52.508.txt'

In [ ]:
len(hash2msg)

In [68]:
# compute the best partition
import datetime
import community as community_louvain
starttime = datetime.datetime.now()
#long running
partition = community_louvain.best_partition(gg.to_undirected())
#do something other
endtime = datetime.datetime.now()
print("Finished the computation of community discovery. Execution time:{:d}".format((endtime - starttime).seconds))

ModuleNotFoundError: No module named 'community'

In [ ]:
communities={}
max_partition=0
for i in partition:
    if partition[i]>max_partition:
        max_partition=partition[i]
        
for i in range(max_partition+1):
    communities[i]=nx.DiGraph()
for e in gg.edges:
#     if partition[e[0]]==partition[e[1]]:
    communities[partition[e[0]]].add_edge(e[0],e[1])
    communities[partition[e[1]]].add_edge(e[0],e[1])

In [ ]:
max_partition

In [ ]:
def attack_edge_flag(msg):
    attack_edge_type=[
            '142.20.56.204',
        'lsass.exe',
        '142.20.61.130',
        '132.197.158.98',
        'Credentials',
    ]
    attack_edge_type=attack_node_set_h201
    flag=False
    for i in attack_edge_type:
        if i in msg:
            flag=True
    return flag

In [ ]:
def find_time_window(edge,tw_dic):
    for t in tw_dic:
#         print(t['start'])
        if edge['time']>=t['start'] and edge['time']<=t['end']:
#             print(t['file'])
            return t['file']

In [ ]:
max_edge_count=-99
max_index=-99
max_node_count=0

graph_index=0
for c in communities:
    file_set=set()
    if len(communities[c].edges)>max_edge_count:
        max_edge_count=len(communities[c].edges)
        max_index=graph_index
        max_node_count=len(communities[c].nodes)
    for e in communities[c].edges:    
        try:
            temp_edge=gg.edges[e]
            file_set.add(find_time_window(temp_edge,attack_dic))
        except:
            pass    
    print(f"{graph_index=}")
    print(f"file_set：")
    print(*file_set, sep = "\n")
    print(f"{file_set=}")
    graph_index+=1
    
print(f"{max_index=}")
print(f"{max_edge_count=}")
print(f"{max_node_count=}") 

In [ ]:
file_sub_list={
    '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 12:55:28.286~2019-09-23 13:09:50.95.txt', 
    '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 13:10:24.95~2019-09-23 13:24:56.43.txt', 
    '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 12:38:24.95~2019-09-23 12:54:14.286.txt', 
    '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 11:38:40.698~2019-09-23 11:53:39.57.txt', 
    '/home/shahidul2k9/data/optc/test/graph_9_23_h201/2019-09-23 11:23:44.136~2019-09-23 11:38:30.698.txt'
}

In [ ]:
original_edges_count=0

for path in tqdm(file_sub_list):

    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            edge_list.append(line)

        original_edges_count+=len(edge_list)

        print(f"{original_edges_count=}")

        

In [ ]:
from graphviz import Digraph


graph_index=0

total_nodes=0
total_edges=0
attck_nodes_set_visual=set()
for c in communities:
    dot = Digraph(name="MyPicture", comment="the test", format="pdf")
    dot.graph_attr['rankdir'] = 'LR'
    
    total_nodes+=len(communities[c].nodes)
    total_edges+=len(communities[c].edges)
    
    subgraph_loss_sum=0
    attack_node_count=0
    attack_edge_count=0
    for e in communities[c].edges:
        try:
            temp_edge=gg.edges[e]
            srcnode=e['srcnode']
            dstnode=e['dstnode']
        except:
            pass        

        if True:
            subgraph_loss_sum+=temp_edge['loss']
   

            if "'subject': '" in temp_edge['srcmsg']:
                src_shape='box'
            elif "'file': '" in temp_edge['srcmsg']:
                src_shape='oval'
            elif "'netflow': '" in temp_edge['srcmsg']:
                src_shape='diamond'
                
            src_shape='box'
            if attack_edge_flag(temp_edge['srcmsg']):
                src_node_color='red'
                total_nodes+=1
            else:
                src_node_color='blue'
                
            src_node_color='blue'
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['srcmsg']))),
                     label=str(replace_path_name(temp_edge['srcmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['srcmsg'])))])), 
                     color=src_node_color,
                     shape = src_shape)


            if "'subject': '" in temp_edge['dstmsg']:
                dst_shape='box'
            elif "'file': '" in temp_edge['dstmsg']:
                dst_shape='oval'
            elif "'netflow': '" in temp_edge['dstmsg']:
                dst_shape='diamond'

                    
            if "->" in temp_edge['dstmsg']:
                dst_shape='diamond'
            else:
                dst_shape='oval'
                
                
            if attack_edge_flag(temp_edge['dstmsg']):
                dst_node_color='red'
                total_nodes+=1
            else:
                dst_node_color='blue'
            dst_node_color='blue'
            
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['dstmsg']))),
                     label=str(replace_path_name(temp_edge['dstmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['dstmsg'])))])), 
                     color=dst_node_color,
                     shape = dst_shape)


    #         edgeindex=tensor_find(test_data.msg[i][16:-16],1)

    
            temp_edge_visual=replace_path_name(temp_edge['srcmsg'])+','+replace_path_name(temp_edge['dstmsg'])+','+str(temp_edge['time'])
            temp_edge_hash_val=hashgen(temp_edge_visual)
            
            if temp_edge_hash_val in edges_attack_hashset_h201:
                edge_color='red'
                attck_nodes_set_visual.add(replace_path_name(temp_edge['srcmsg']))
                attck_nodes_set_visual.add(replace_path_name(temp_edge['dstmsg']))
                attack_edge_count+=1
            else:
                edge_color='blue'
            
#             if attack_edge_flag(temp_edge['srcmsg']) and attack_edge_flag(temp_edge['dstmsg']):
#                 edge_color='red'
#                 attack_edge_count+=1
#             else:
#                 edge_color='blue'
                
                
            dot.edge(str(hashgen(replace_path_name(temp_edge['srcmsg']))),str(hashgen(replace_path_name(temp_edge['dstmsg']))), label= temp_edge['edge_type'] , color=edge_color)#+ "  loss: "+str(temp_edge['loss']) + "  time: "+str(temp_edge['time'])

    #         dot.edge(str(srcnode), str(dstnode), label= temp_edge['edge_type']+ "  loss: "+str((temp_edge['loss'])) + "  time: "+str(temp_edge['time']) , color='red')


#     if len(communities[c].edges)<2:
#         graph_index+=1
#         continue
#     if len(communities[c].edges)>1000:
#         graph_index+=1
#         continue
#         print(f"edge num:{len(communities[c].edges)}")
        
    print("Start to render the figures···")
    
    dot.render('./graph_visual_h201/subgraph_'+str(graph_index), view=False)
    print("subgraph loss:",(subgraph_loss_sum/len(communities[c].edges)))
    print("graph_index:",graph_index)
    print("edge_count:",len(communities[c].edges))
    print("node_count:",len(communities[c].nodes))
    print(f"{attack_node_count=}")
    print(f"{attack_edge_count=}")
    graph_index+=1

print(f"avg edges:{total_edges/len(communities)}")
print(f"avg nodes:{total_nodes/len(communities)}")

In [ ]:
attck_nodes_set_visual

In [ ]:
len(attck_nodes_set_visual)

In [ ]:
temp_edge_visual

## h402

In [ ]:
attack_list=[
 '/home/shahidul2k9/data/optc/test/graph_9_23_h402/2019-09-23 13:10:24.429~2019-09-23 13:25:09.374.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h402/2019-09-23 13:10:24.429~2019-09-23 13:25:09.374.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h402/2019-09-23 13:25:20.374~2019-09-23 13:40:21.268.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h402/2019-09-23 13:40:16.268~2019-09-23 13:55:31.310.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h402/2019-09-23 13:55:12.31~2019-09-23 14:10:58.200.txt',
            ]


In [ ]:
from datetime import datetime, timezone
attack_dic=[]
for a in attack_list:
    temp_dic={}
    aa=a.split("/")[-1]
    sp=aa.split('~')
    start_timestamp=sp[0].split('.')[0]
    end_timestamp=sp[1].split('.')[0]
    temp_dic['start']=datetime_to_timestamp_US(start_timestamp)
    temp_dic['end']=datetime_to_timestamp_US(end_timestamp)
    temp_dic['file']=a
    attack_dic.append(temp_dic)
    print(temp_dic)

In [ ]:
replace_dic={
    '.pyc':'*.pyc',
    '.dll':'*.dll',
    '.DLL':'*.DLL',
}

def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    if '->' in path_name:
        if 'outbound' in path_name:
            msg=re.findall("->(.*?):",path_name)[0]
            return msg
        elif 'inbound' in path_name:
            msg=re.findall("#(.*?):",path_name)[0]
            return msg
    return path_name

In [ ]:

original_edges_count=0
hash2msg={}
graphs=[]
gg=nx.DiGraph()
count=0
# file_list=os.listdir("./test_day_data4_10_emb100/")
for path in tqdm(attack_list):
#     print(path)
    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            count+=1
            l=line.strip()
            jdata=eval(l)
#             temp_key=jdata['srcmsg']+jdata['dstmsg']+jdata['edge_type']
#             if temp_key in train_edge_set:
#                 jdata['loss']=(jdata['loss']-train_edge_set[temp_key]) if jdata['loss']>=train_edge_set[temp_key] else 0  
#             jdata['loss']=abs(jdata['loss']-train_edge_set[temp_key])  if temp_key in train_edge_set else jdata['loss']
            edge_list.append(jdata)
            
        edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True) 
        original_edges_count+=len(edge_list)
        
        loss_list=[]
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean=mean(loss_list)
        loss_std=std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr=loss_mean+1.5*loss_std
#         thr=-99
        print("thr:",thr)
        for e in edge_list:
            if e['loss']>thr:    
#             if True:  
#                 if "'/home/admin/profile'" in e['srcmsg'] or " '/home/admin/profile'" in e['dstmsg']:
#                     print(e['srcmsg'])
#                     print(e['dstmsg'])
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))),loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
                
                hash2msg[str(hashgen(replace_path_name(e['srcmsg'])))]=replace_path_name(e['srcmsg'])
                hash2msg[str(hashgen(replace_path_name(e['dstmsg'])))]=replace_path_name(e['dstmsg'])
                

#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
        print(path)
        print("tempg edges:",len(tempg.edges))
        print("tempg nodes:",len(tempg.nodes))
        print("tempg weakly components:",nx.number_weakly_connected_components(tempg))
        
        print("gg edges:",len(gg.edges))
        print("gg nodes:",len(gg.nodes))
        print("gg weakly components:",nx.number_weakly_connected_components(gg))
        print(f"{original_edges_count=}")
#
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

In [ ]:
# compute the best partition
import datetime
import community as community_louvain
starttime = datetime.datetime.now()
#long running
partition = community_louvain.best_partition(gg.to_undirected())
#do something other
endtime = datetime.datetime.now()
print("Finished the computation of community discovery. Execution time:{:d}".format((endtime - starttime).seconds))


communities={}
max_partition=0
for i in partition:
    if partition[i]>max_partition:
        max_partition=partition[i]
        
for i in range(max_partition+1):
    communities[i]=nx.DiGraph()
for e in gg.edges:
#     if partition[e[0]]==partition[e[1]]:
    communities[partition[e[0]]].add_edge(e[0],e[1])
    communities[partition[e[1]]].add_edge(e[0],e[1])
    
print(f"{max_partition=}")

In [ ]:
max_edge_count=-99
max_index=-99
max_node_count=0

graph_index=0
for c in communities:
    file_set=set()
    if len(communities[c].edges)>max_edge_count:
        max_edge_count=len(communities[c].edges)
        max_index=graph_index
        max_node_count=len(communities[c].nodes)
    for e in communities[c].edges:    
        try:
            temp_edge=gg.edges[e]
            file_set.add(find_time_window(temp_edge,attack_dic))
        except:
            pass    
    print(f"{graph_index=}")
    print(f"file_set：")
    print(*file_set, sep = "\n")
    print(f"{file_set=}")
    graph_index+=1
    
print(f"{max_index=}")
print(f"{max_edge_count=}")
print(f"{max_node_count=}") 

In [ ]:
file_sub_list={'/home/shahidul2k9/data/optc/test/graph_9_23_h402/2019-09-23 13:25:20.374~2019-09-23 13:40:21.268.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_23_h402/2019-09-23 13:10:24.429~2019-09-23 13:25:09.374.txt'}




original_edges_count=0

for path in tqdm(file_sub_list):

    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            edge_list.append(line)

        original_edges_count+=len(edge_list)

        print(f"{original_edges_count=}")
#
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

In [ ]:
len(communities)

In [ ]:

from graphviz import Digraph


graph_index=0

total_nodes=0
total_edges=0
attck_nodes_set_visual=set()
for c in communities:

    dot = Digraph(name="MyPicture", comment="the test", format="pdf")
    dot.graph_attr['rankdir'] = 'LR'
    # dot.node(name='a', label='wo', color='purple')
    # dot.node(name='b', label='niu', color='purple')
    # dot.node(name='c', label='che', color='purple')
    
    total_nodes+=len(communities[c].nodes)
    total_edges+=len(communities[c].edges)
    
    subgraph_loss_sum=0
    attack_node_count=0
    attack_edge_count=0
    for e in communities[c].edges:
        try:
            temp_edge=gg.edges[e]
            srcnode=e['srcnode']
            dstnode=e['dstnode']
        except:
            pass        

        if True:
            subgraph_loss_sum+=temp_edge['loss']

    #     g.add_edge(srcnode,dstnode,srcmsg=node2msg[indexid2nodeid[srcnode]],dstmsg=node2msg[indexid2nodeid[dstnode]],loss=df['loss'][i])


    #         dot.node( name=str(srcnode),label=str(node_index_id2msg_mal[srcnode]), color='purple',shape = 'box')
            if "'subject': '" in temp_edge['srcmsg']:
                src_shape='box'
            elif "'file': '" in temp_edge['srcmsg']:
                src_shape='oval'
            elif "'netflow': '" in temp_edge['srcmsg']:
                src_shape='diamond'
                
            src_shape='box'
            if attack_edge_flag(temp_edge['srcmsg']):
                src_node_color='red'
                total_nodes+=1
            else:
                src_node_color='blue'
                
            src_node_color='blue'
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['srcmsg']))),
                     label=str(replace_path_name(temp_edge['srcmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['srcmsg'])))])), 
                     color=src_node_color,
                     shape = src_shape)


            if "'subject': '" in temp_edge['dstmsg']:
                dst_shape='box'
            elif "'file': '" in temp_edge['dstmsg']:
                dst_shape='oval'
            elif "'netflow': '" in temp_edge['dstmsg']:
                dst_shape='diamond'

                    
            if "->" in temp_edge['dstmsg']:
                dst_shape='diamond'
            else:
                dst_shape='oval'
                
                
            if attack_edge_flag(temp_edge['dstmsg']):
                dst_node_color='red'
                total_nodes+=1
            else:
                dst_node_color='blue'
            dst_node_color='blue'
            
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['dstmsg']))),
                     label=str(replace_path_name(temp_edge['dstmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['dstmsg'])))])), 
                     color=dst_node_color,
                     shape = dst_shape)


    #         edgeindex=tensor_find(test_data.msg[i][16:-16],1)

    
            temp_edge_visual=replace_path_name(temp_edge['srcmsg'])+','+replace_path_name(temp_edge['dstmsg'])+','+str(temp_edge['time'])
            temp_edge_hash_val=hashgen(temp_edge_visual)
            
            if temp_edge_hash_val in edges_attack_hashset_h402:
                edge_color='red'
                attck_nodes_set_visual.add(replace_path_name(temp_edge['srcmsg']))
                attck_nodes_set_visual.add(replace_path_name(temp_edge['dstmsg']))
                attack_edge_count+=1
            else:
                edge_color='blue'
            
#             if attack_edge_flag(temp_edge['srcmsg']) and attack_edge_flag(temp_edge['dstmsg']):
#                 edge_color='red'
#                 attack_edge_count+=1
#             else:
#                 edge_color='blue'
                
                
            dot.edge(str(hashgen(replace_path_name(temp_edge['srcmsg']))),str(hashgen(replace_path_name(temp_edge['dstmsg']))), label= temp_edge['edge_type'] , color=edge_color)#+ "  loss: "+str(temp_edge['loss']) + "  time: "+str(temp_edge['time'])

    #         dot.edge(str(srcnode), str(dstnode), label= temp_edge['edge_type']+ "  loss: "+str((temp_edge['loss'])) + "  time: "+str(temp_edge['time']) , color='red')


    if len(communities[c].edges)<2:
        graph_index+=1
        continue
    if len(communities[c].edges)>1000:
        graph_index+=1
        continue
        print(f"edge num:{len(communities[c].edges)}，skip rendering")
        
    print("Start to reder the figures···")
    
    dot.render('./graph_visual_h402/subgraph_'+str(graph_index), view=False)
    print("subgraph loss:",(subgraph_loss_sum/len(communities[c].edges)))
    print("graph_index:",graph_index)
    print("edge_count:",len(communities[c].edges))
    print("node_count:",len(communities[c].nodes))
    print(f"{attack_node_count=}")
    print(f"{attack_edge_count=}")
    graph_index+=1

print(f"avg edges:{total_edges/len(communities)}")
print(f"avg nodes:{total_nodes/len(communities)}")

In [ ]:
attck_nodes_set_visual

## h660

In [ ]:
attack_list=[
    
    '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 09:54:08.697~2019-09-23 10:09:11.523.txt',
'/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 13:27:28.512~2019-09-23 13:42:30.682.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 13:42:24.682~2019-09-23 13:57:57.566.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 13:57:20.566~2019-09-23 14:12:59.139.txt',
            ]


In [ ]:
from datetime import datetime, timezone
attack_dic=[]
for a in attack_list:
    temp_dic={}
    aa=a.split("/")[-1]
    sp=aa.split('~')
    start_timestamp=sp[0].split('.')[0]
    end_timestamp=sp[1].split('.')[0]
    temp_dic['start']=datetime_to_timestamp_US(start_timestamp)
    temp_dic['end']=datetime_to_timestamp_US(end_timestamp)
    temp_dic['file']=a
    attack_dic.append(temp_dic)
    print(temp_dic)

In [ ]:
replace_dic={
    '.pyc':'*.pyc',
#     '.dll':'*.dll',
#     '.DLL':'*.DLL',
}

def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    if '->' in path_name:
        if 'outbound' in path_name:
            msg=re.findall("->(.*?):",path_name)[0]
            return msg
        elif 'inbound' in path_name:
            msg=re.findall("#(.*?):",path_name)[0]
            return msg
    return path_name

In [ ]:

original_edges_count=0
hash2msg={}
graphs=[]
gg=nx.DiGraph()
count=0
# file_list=os.listdir("./test_day_data4_10_emb100/")
for path in tqdm(attack_list):
#     print(path)
    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            count+=1
            l=line.strip()
            jdata=eval(l)
#             temp_key=jdata['srcmsg']+jdata['dstmsg']+jdata['edge_type']
#             if temp_key in train_edge_set:
#                 jdata['loss']=(jdata['loss']-train_edge_set[temp_key]) if jdata['loss']>=train_edge_set[temp_key] else 0  
#             jdata['loss']=abs(jdata['loss']-train_edge_set[temp_key])  if temp_key in train_edge_set else jdata['loss']
            edge_list.append(jdata)
            
        edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True) 
        original_edges_count+=len(edge_list)
        
        loss_list=[]
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean=mean(loss_list)
        loss_std=std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr=loss_mean+1.5*loss_std
#         thr=-99
        print("thr:",thr)
        for e in edge_list:
            if e['loss']>thr:    
#             if True:  
#                 if "'/home/admin/profile'" in e['srcmsg'] or " '/home/admin/profile'" in e['dstmsg']:
#                     print(e['srcmsg'])
#                     print(e['dstmsg'])
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))),loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
                
                hash2msg[str(hashgen(replace_path_name(e['srcmsg'])))]=replace_path_name(e['srcmsg'])
                hash2msg[str(hashgen(replace_path_name(e['dstmsg'])))]=replace_path_name(e['dstmsg'])
                

#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
        print(path)
        print("tempg edges:",len(tempg.edges))
        print("tempg nodes:",len(tempg.nodes))
        print("tempg weakly components:",nx.number_weakly_connected_components(tempg))
        
        print("gg edges:",len(gg.edges))
        print("gg nodes:",len(gg.nodes))
        print("gg weakly components:",nx.number_weakly_connected_components(gg))
        print(f"{original_edges_count=}")
#
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

In [ ]:
# compute the best partition
import datetime
import community as community_louvain
starttime = datetime.datetime.now()
#long running
partition = community_louvain.best_partition(gg.to_undirected())
#do something other
endtime = datetime.datetime.now()
print("Finished the computation of community discovery. Execution time:{:d}".format((endtime - starttime).seconds))


communities={}
max_partition=0
for i in partition:
    if partition[i]>max_partition:
        max_partition=partition[i]
        
for i in range(max_partition+1):
    communities[i]=nx.DiGraph()
for e in gg.edges:
#     if partition[e[0]]==partition[e[1]]:
    communities[partition[e[0]]].add_edge(e[0],e[1])
    communities[partition[e[1]]].add_edge(e[0],e[1])
    
print(f"{max_partition=}")

In [ ]:

max_edge_count=-99
max_index=-99
max_node_count=0

graph_index=0
for c in communities:
    file_set=set()
    if len(communities[c].edges)>max_edge_count:
        max_edge_count=len(communities[c].edges)
        max_index=graph_index
        max_node_count=len(communities[c].nodes)
    for e in communities[c].edges:    
        try:
            temp_edge=gg.edges[e]
            file_set.add(find_time_window(temp_edge,attack_dic))
        except:
            pass    
    print(f"{graph_index=}")
    print(f"file_set：")
    print(*file_set, sep = "\n")
    print(f"{file_set=}")
    graph_index+=1
    
print(f"{max_index=}")
print(f"{max_edge_count=}")
print(f"{max_node_count=}") 

In [ ]:
file_sub_list={'/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 13:27:28.512~2019-09-23 13:42:30.682.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 13:42:24.682~2019-09-23 13:57:57.566.txt',
                '/home/shahidul2k9/data/optc/test/graph_9_23_h660/2019-09-23 13:57:20.566~2019-09-23 14:12:59.139.txt'}



original_edges_count=0

for path in tqdm(file_sub_list):

    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            edge_list.append(line)

        original_edges_count+=len(edge_list)

        print(f"{original_edges_count=}")
#
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

In [ ]:

from graphviz import Digraph


graph_index=0

total_nodes=0
total_edges=0
attck_nodes_set_visual=set()
for c in communities:
    dot = Digraph(name="MyPicture", comment="the test", format="pdf")
    dot.graph_attr['rankdir'] = 'LR'
    # dot.node(name='a', label='wo', color='purple')
    # dot.node(name='b', label='niu', color='purple')
    # dot.node(name='c', label='che', color='purple')
    
    total_nodes+=len(communities[c].nodes)
    total_edges+=len(communities[c].edges)
    
    subgraph_loss_sum=0
    attack_node_count=0
    attack_edge_count=0
    for e in communities[c].edges:
        try:
            temp_edge=gg.edges[e]
            srcnode=e['srcnode']
            dstnode=e['dstnode']
        except:
            pass        

        if True:
            subgraph_loss_sum+=temp_edge['loss']

    #     g.add_edge(srcnode,dstnode,srcmsg=node2msg[indexid2nodeid[srcnode]],dstmsg=node2msg[indexid2nodeid[dstnode]],loss=df['loss'][i])


    #         dot.node( name=str(srcnode),label=str(node_index_id2msg_mal[srcnode]), color='purple',shape = 'box')
            if "'subject': '" in temp_edge['srcmsg']:
                src_shape='box'
            elif "'file': '" in temp_edge['srcmsg']:
                src_shape='oval'
            elif "'netflow': '" in temp_edge['srcmsg']:
                src_shape='diamond'
                
            src_shape='box'
            if attack_edge_flag(temp_edge['srcmsg']):
                src_node_color='red'
                total_nodes+=1
            else:
                src_node_color='blue'
                
            src_node_color='blue'
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['srcmsg']))),
                     label=str(replace_path_name(temp_edge['srcmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['srcmsg'])))])), 
                     color=src_node_color,
                     shape = src_shape)


            if "'subject': '" in temp_edge['dstmsg']:
                dst_shape='box'
            elif "'file': '" in temp_edge['dstmsg']:
                dst_shape='oval'
            elif "'netflow': '" in temp_edge['dstmsg']:
                dst_shape='diamond'

                    
            if "->" in temp_edge['dstmsg']:
                dst_shape='diamond'
            else:
                dst_shape='oval'
                
                
            if attack_edge_flag(temp_edge['dstmsg']):
                dst_node_color='red'
                total_nodes+=1
            else:
                dst_node_color='blue'
            dst_node_color='blue'
            
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['dstmsg']))),
                     label=str(replace_path_name(temp_edge['dstmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['dstmsg'])))])), 
                     color=dst_node_color,
                     shape = dst_shape)


    #         edgeindex=tensor_find(test_data.msg[i][16:-16],1)

    
            temp_edge_visual=replace_path_name(temp_edge['srcmsg'])+','+replace_path_name(temp_edge['dstmsg'])+','+str(temp_edge['time'])
            temp_edge_hash_val=hashgen(temp_edge_visual)
            
            if temp_edge_hash_val in edges_attack_hashset_h660:
                edge_color='red'
                attck_nodes_set_visual.add(replace_path_name(temp_edge['srcmsg']))
                attck_nodes_set_visual.add(replace_path_name(temp_edge['dstmsg']))
                attack_edge_count+=1
            else:
                edge_color='blue'
            
#             if attack_edge_flag(temp_edge['srcmsg']) and attack_edge_flag(temp_edge['dstmsg']):
#                 edge_color='red'
#                 attack_edge_count+=1
#             else:
#                 edge_color='blue'
                
                
            dot.edge(str(hashgen(replace_path_name(temp_edge['srcmsg']))),str(hashgen(replace_path_name(temp_edge['dstmsg']))), label= temp_edge['edge_type'] , color=edge_color)#+ "  loss: "+str(temp_edge['loss']) + "  time: "+str(temp_edge['time'])

    #         dot.edge(str(srcnode), str(dstnode), label= temp_edge['edge_type']+ "  loss: "+str((temp_edge['loss'])) + "  time: "+str(temp_edge['time']) , color='red')


#     if len(communities[c].edges)<2:
#         continue
#     if len(communities[c].edges)>1000:
#         continue
#         print(f"edge num:{len(communities[c].edges)}，skip rendering")
        
    print("Start to render the figures···")
    
    dot.render('./graph_visual_h660/subgraph_'+str(graph_index), view=False)
    print("subgraph loss:",(subgraph_loss_sum/len(communities[c].edges)))
    print("graph_index:",graph_index)
    print("edge_count:",len(communities[c].edges))
    print("node_count:",len(communities[c].nodes))
    print(f"{attack_node_count=}")
    print(f"{attack_edge_count=}")
    graph_index+=1

print(f"avg edges:{total_edges/len(communities)}")
print(f"avg nodes:{total_nodes/len(communities)}")

In [ ]:
attck_nodes_set_visual

## h501

In [155]:
attack_list=[
 'graph_9_24_h501/2019-09-24 10:15:28.241~2019-09-24 10:30:00.201.txt',
 'graph_9_24_h501/2019-09-24 10:30:24.201~2019-09-24 10:45:02.7.txt',
 'graph_9_24_h501/2019-09-24 10:45:20.7~2019-09-24 11:00:31.385.txt',
 'graph_9_24_h501/2019-09-24 11:00:16.385~2019-09-24 11:16:09.755.txt',
 'graph_9_24_h501/2019-09-24 11:15:12.755~2019-09-24 11:31:14.287.txt',
 'graph_9_24_h501/2019-09-24 11:32:16.287~2019-09-24 11:46:31.541.txt',
    
 'graph_9_24_h501/2019-09-24 13:04:00.804~2019-09-24 13:17:29.451.txt',
 'graph_9_24_h501/2019-09-24 13:18:56.451~2019-09-24 13:32:46.454.txt',
 'graph_9_24_h501/2019-09-24 13:33:52.454~2019-09-24 13:48:02.493.txt',
            ]


In [156]:
from datetime import datetime, timezone
attack_dic=[]
for a in attack_list:
    temp_dic={}
    aa=a.split("/")[-1]
    sp=aa.split('~')
    start_timestamp=sp[0].split('.')[0]
    end_timestamp=sp[1].split('.')[0]
    temp_dic['start']=datetime_to_timestamp_US(start_timestamp)
    temp_dic['end']=datetime_to_timestamp_US(end_timestamp)
    temp_dic['file']=a
    attack_dic.append(temp_dic)
    print(temp_dic)

ValueError: time data '2019-09-24 10:15:28' does not match format '%Y-%m-%dT%H:%M:%S'

In [ ]:
replace_dic={
    '.pyc':'*.pyc',
    '.dll':'*.dll',
    '.DLL':'*.DLL',
}

def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    if '->' in path_name:
        if 'outbound' in path_name:
            msg=re.findall("->(.*?):",path_name)[0]
            return msg
        elif 'inbound' in path_name:
            msg=re.findall("#(.*?):",path_name)[0]
            return msg
    return path_name

In [ ]:

original_edges_count=0
hash2msg={}
graphs=[]
gg=nx.DiGraph()
count=0
# file_list=os.listdir("./test_day_data4_10_emb100/")
for path in tqdm(attack_list):
#     print(path)
    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            count+=1
            l=line.strip()
            jdata=eval(l)
#             temp_key=jdata['srcmsg']+jdata['dstmsg']+jdata['edge_type']
#             if temp_key in train_edge_set:
#                 jdata['loss']=(jdata['loss']-train_edge_set[temp_key]) if jdata['loss']>=train_edge_set[temp_key] else 0  
#             jdata['loss']=abs(jdata['loss']-train_edge_set[temp_key])  if temp_key in train_edge_set else jdata['loss']
            edge_list.append(jdata)
            
        edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True) 
        original_edges_count+=len(edge_list)
        
        loss_list=[]
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean=mean(loss_list)
        loss_std=std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr=loss_mean+2.5*loss_std
#         thr=-99
        print("thr:",thr)
        for e in edge_list:
            if e['loss']>thr:    
#             if True:  
#                 if "'/home/admin/profile'" in e['srcmsg'] or " '/home/admin/profile'" in e['dstmsg']:
#                     print(e['srcmsg'])
#                     print(e['dstmsg'])
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))),loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
                
                hash2msg[str(hashgen(replace_path_name(e['srcmsg'])))]=replace_path_name(e['srcmsg'])
                hash2msg[str(hashgen(replace_path_name(e['dstmsg'])))]=replace_path_name(e['dstmsg'])
                

#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
        print(path)
        print("tempg edges:",len(tempg.edges))
        print("tempg nodes:",len(tempg.nodes))
        print("tempg weakly components:",nx.number_weakly_connected_components(tempg))
        
        print("gg edges:",len(gg.edges))
        print("gg nodes:",len(gg.nodes))
        print("gg weakly components:",nx.number_weakly_connected_components(gg))
        print(f"{original_edges_count=}")
#
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

In [ ]:
# compute the best partition
import datetime
import community as community_louvain
starttime = datetime.datetime.now()
#long running
partition = community_louvain.best_partition(gg.to_undirected())
#do something other
endtime = datetime.datetime.now()
print("Finished the computation of community discovery. Execution time:{:d}".format((endtime - starttime).seconds))


communities={}
max_partition=0
for i in partition:
    if partition[i]>max_partition:
        max_partition=partition[i]
        
for i in range(max_partition+1):
    communities[i]=nx.DiGraph()
for e in gg.edges:
#     if partition[e[0]]==partition[e[1]]:
    communities[partition[e[0]]].add_edge(e[0],e[1])
    communities[partition[e[1]]].add_edge(e[0],e[1])
    
print(f"{max_partition=}")

In [ ]:
max_edge_count=-99
max_index=-99
max_node_count=0

graph_index=0
for c in communities:
    file_set=set()
    if len(communities[c].edges)>max_edge_count:
        max_edge_count=len(communities[c].edges)
        max_index=graph_index
        max_node_count=len(communities[c].nodes)
    for e in communities[c].edges:    
        try:
            temp_edge=gg.edges[e]
            file_set.add(find_time_window(temp_edge,attack_dic))
        except:
            pass    
    print(f"{graph_index=}")
    print(f"file_set：")
    print(*file_set, sep = "\n")
    print(f"{file_set=}")
    graph_index+=1
    
print(f"{max_index=}")
print(f"{max_edge_count=}")
print(f"{max_node_count=}") 

In [154]:
file_sub_list={'/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 10:45:20.7~2019-09-24 11:00:31.385.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 11:00:16.385~2019-09-24 11:16:09.755.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 10:15:28.241~2019-09-24 10:30:00.201.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 11:15:12.755~2019-09-24 11:31:14.287.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 11:32:16.287~2019-09-24 11:46:31.541.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 10:30:24.201~2019-09-24 10:45:02.7.txt',
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 10:15:28.241~2019-09-24 10:30:00.201.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 13:04:00.804~2019-09-24 13:17:29.451.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 11:15:12.755~2019-09-24 11:31:14.287.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 11:32:16.287~2019-09-24 11:46:31.541.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 13:33:52.454~2019-09-24 13:48:02.493.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 10:30:24.201~2019-09-24 10:45:02.7.txt', 
               '/home/shahidul2k9/data/optc/test/graph_9_24_h501/2019-09-24 13:18:56.451~2019-09-24 13:32:46.454.txt'}

original_edges_count=0

for path in tqdm(file_sub_list):

    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            edge_list.append(line)

        original_edges_count+=len(edge_list)

        print(f"{original_edges_count=}")
#
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

KeyboardInterrupt: 

In [153]:
from graphviz import Digraph


graph_index=0

total_nodes=0
total_edges=0
attck_nodes_set_visual=set()
for c in communities:
    dot = Digraph(name="MyPicture", comment="the test", format="pdf")
    dot.graph_attr['rankdir'] = 'LR'
    # dot.node(name='a', label='wo', color='purple')
    # dot.node(name='b', label='niu', color='purple')
    # dot.node(name='c', label='che', color='purple')
    
    total_nodes+=len(communities[c].nodes)
    total_edges+=len(communities[c].edges)
    
    subgraph_loss_sum=0
    attack_node_count=0
    attack_edge_count=0
    for e in communities[c].edges:
        try:
            temp_edge=gg.edges[e]
            srcnode=e['srcnode']
            dstnode=e['dstnode']
        except:
            pass        

        if True:
            subgraph_loss_sum+=temp_edge['loss']

    #     g.add_edge(srcnode,dstnode,srcmsg=node2msg[indexid2nodeid[srcnode]],dstmsg=node2msg[indexid2nodeid[dstnode]],loss=df['loss'][i])


    #         dot.node( name=str(srcnode),label=str(node_index_id2msg_mal[srcnode]), color='purple',shape = 'box')
            if "'subject': '" in temp_edge['srcmsg']:
                src_shape='box'
            elif "'file': '" in temp_edge['srcmsg']:
                src_shape='oval'
            elif "'netflow': '" in temp_edge['srcmsg']:
                src_shape='diamond'
                
            src_shape='box'
            if attack_edge_flag(temp_edge['srcmsg']):
                src_node_color='red'
                total_nodes+=1
            else:
                src_node_color='blue'
                
            src_node_color='blue'
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['srcmsg']))),
                     label=str(replace_path_name(temp_edge['srcmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['srcmsg'])))])), 
                     color=src_node_color,
                     shape = src_shape)


            if "'subject': '" in temp_edge['dstmsg']:
                dst_shape='box'
            elif "'file': '" in temp_edge['dstmsg']:
                dst_shape='oval'
            elif "'netflow': '" in temp_edge['dstmsg']:
                dst_shape='diamond'

                    
            if "->" in temp_edge['dstmsg']:
                dst_shape='diamond'
            else:
                dst_shape='oval'
                
                
            if attack_edge_flag(temp_edge['dstmsg']):
                dst_node_color='red'
                total_nodes+=1
            else:
                dst_node_color='blue'
            dst_node_color='blue'
            
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['dstmsg']))),
                     label=str(replace_path_name(temp_edge['dstmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['dstmsg'])))])), 
                     color=dst_node_color,
                     shape = dst_shape)


    #         edgeindex=tensor_find(test_data.msg[i][16:-16],1)
    
            temp_edge_visual=replace_path_name(temp_edge['srcmsg'])+','+replace_path_name(temp_edge['dstmsg'])+','+str(temp_edge['time'])
            temp_edge_hash_val=hashgen(temp_edge_visual)
            
            if temp_edge_hash_val in edges_attack_hashset_h501:
                edge_color='red'
                attck_nodes_set_visual.add(replace_path_name(temp_edge['srcmsg']))
                attck_nodes_set_visual.add(replace_path_name(temp_edge['dstmsg']))
                attack_edge_count+=1
            else:
                edge_color='blue'
            
#             if attack_edge_flag(temp_edge['srcmsg']) and attack_edge_flag(temp_edge['dstmsg']):
#                 edge_color='red'
#                 attack_edge_count+=1
#             else:
#                 edge_color='blue'
                
                
            dot.edge(str(hashgen(replace_path_name(temp_edge['srcmsg']))),str(hashgen(replace_path_name(temp_edge['dstmsg']))), label= temp_edge['edge_type'] , color=edge_color)#+ "  loss: "+str(temp_edge['loss']) + "  time: "+str(temp_edge['time'])

    #         dot.edge(str(srcnode), str(dstnode), label= temp_edge['edge_type']+ "  loss: "+str((temp_edge['loss'])) + "  time: "+str(temp_edge['time']) , color='red')


#     if len(communities[c].edges)<2:
#         continue
#     if len(communities[c].edges)>1000:
#         continue
#         print(f"edge num:{len(communities[c].edges)}，skip rendering")
        
    print("Start to render the figures···")
    
    dot.render('./graph_visual_h501/subgraph_'+str(graph_index), view=False)
    print("subgraph loss:",(subgraph_loss_sum/len(communities[c].edges)))
    print("graph_index:",graph_index)
    print("edge_count:",len(communities[c].edges))
    print("node_count:",len(communities[c].nodes))
    print(f"{attack_node_count=}")
    print(f"{attack_edge_count=}")
    graph_index+=1

print(f"avg edges:{total_edges/len(communities)}")
print(f"avg nodes:{total_nodes/len(communities)}")

NameError: name 'communities' is not defined

In [ ]:
attck_nodes_set_visual

## h051

In [ ]:
attack_list=[
    '/home/shahidul2k9/data/optc/test/graph_9_25_h051/2019-09-25 09:11:28.846~2019-09-25 09:25:34.393.txt',
    '/home/shahidul2k9/data/optc/test/graph_9_25_h051/2019-09-25 09:26:24.393~2019-09-25 09:40:36.393.txt',
'/home/shahidul2k9/data/optc/test/graph_9_25_h051/2019-09-25 10:26:08.397~2019-09-25 10:41:40.247.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_25_h051/2019-09-25 10:41:04.247~2019-09-25 10:56:56.92.txt',
 '/home/shahidul2k9/data/optc/test/graph_9_25_h051/2019-09-25 10:56:00.92~2019-09-25 11:12:03.608.txt',
    '/home/shahidul2k9/data/optc/test/graph_9_25_h051/2019-09-25 14:16:32.762~2019-09-25 14:32:02.764.txt',
            ]


In [ ]:
from datetime import datetime, timezone
attack_dic=[]
for a in attack_list:
    temp_dic={}
    aa=a.split("/")[-1]
    sp=aa.split('~')
    start_timestamp=sp[0].split('.')[0]
    end_timestamp=sp[1].split('.')[0]
    temp_dic['start']=datetime_to_timestamp_US(start_timestamp)
    temp_dic['end']=datetime_to_timestamp_US(end_timestamp)
    temp_dic['file']=a
    attack_dic.append(temp_dic)
    print(temp_dic)

In [ ]:
replace_dic={
    '.pyc':'*.pyc',
    '.dll':'*.dll',
    '.DLL':'*.DLL',
}

def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    if '->' in path_name:
        if 'outbound' in path_name:
            msg=re.findall("->(.*?):",path_name)[0]
            return msg
        elif 'inbound' in path_name:
            msg=re.findall("#(.*?):",path_name)[0]
            return msg
    return path_name

In [ ]:

original_edges_count=0
hash2msg={}
graphs=[]
gg=nx.DiGraph()
count=0
# file_list=os.listdir("./test_day_data4_10_emb100/")
for path in tqdm(attack_list):
#     print(path)
    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            count+=1
            l=line.strip()
            jdata=eval(l)
#             temp_key=jdata['srcmsg']+jdata['dstmsg']+jdata['edge_type']
#             if temp_key in train_edge_set:
#                 jdata['loss']=(jdata['loss']-train_edge_set[temp_key]) if jdata['loss']>=train_edge_set[temp_key] else 0  
#             jdata['loss']=abs(jdata['loss']-train_edge_set[temp_key])  if temp_key in train_edge_set else jdata['loss']
            edge_list.append(jdata)
            
        edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True) 
        original_edges_count+=len(edge_list)
        
        loss_list=[]
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean=mean(loss_list)
        loss_std=std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr=loss_mean+2.5*loss_std
#         thr=-99
        print("thr:",thr)
        for e in edge_list:
            if e['loss']>thr:    
#             if True:  
#                 if "'/home/admin/profile'" in e['srcmsg'] or " '/home/admin/profile'" in e['dstmsg']:
#                     print(e['srcmsg'])
#                     print(e['dstmsg'])
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))),loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
                
                hash2msg[str(hashgen(replace_path_name(e['srcmsg'])))]=replace_path_name(e['srcmsg'])
                hash2msg[str(hashgen(replace_path_name(e['dstmsg'])))]=replace_path_name(e['dstmsg'])
                

#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
        print(path)
        print("tempg edges:",len(tempg.edges))
        print("tempg nodes:",len(tempg.nodes))
        print("tempg weakly components:",nx.number_weakly_connected_components(tempg))
        
        print("gg edges:",len(gg.edges))
        print("gg nodes:",len(gg.nodes))
        print("gg weakly components:",nx.number_weakly_connected_components(gg))
        print(f"{original_edges_count=}")
#
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

In [ ]:
# compute the best partition
import datetime
import community as community_louvain
starttime = datetime.datetime.now()
#long running
partition = community_louvain.best_partition(gg.to_undirected())
#do something other
endtime = datetime.datetime.now()
print("Finished the computation of community discovery. Execution time:{:d}".format((endtime - starttime).seconds))


communities={}
max_partition=0
for i in partition:
    if partition[i]>max_partition:
        max_partition=partition[i]
        
for i in range(max_partition+1):
    communities[i]=nx.DiGraph()
for e in gg.edges:
#     if partition[e[0]]==partition[e[1]]:
    communities[partition[e[0]]].add_edge(e[0],e[1])
    communities[partition[e[1]]].add_edge(e[0],e[1])
    
print(f"{max_partition=}")

In [ ]:
from graphviz import Digraph


graph_index=0

total_nodes=0
total_edges=0
attck_nodes_set_visual=set()
for c in communities:
    dot = Digraph(name="MyPicture", comment="the test", format="pdf")
    dot.graph_attr['rankdir'] = 'LR'
    # dot.node(name='a', label='wo', color='purple')
    # dot.node(name='b', label='niu', color='purple')
    # dot.node(name='c', label='che', color='purple')
    
    total_nodes+=len(communities[c].nodes)
    total_edges+=len(communities[c].edges)
    
    subgraph_loss_sum=0
    attack_node_count=0
    attack_edge_count=0
    for e in communities[c].edges:
        try:
            temp_edge=gg.edges[e]
            srcnode=e['srcnode']
            dstnode=e['dstnode']
        except:
            pass        

        if True:
            subgraph_loss_sum+=temp_edge['loss']

    #     g.add_edge(srcnode,dstnode,srcmsg=node2msg[indexid2nodeid[srcnode]],dstmsg=node2msg[indexid2nodeid[dstnode]],loss=df['loss'][i])


    #         dot.node( name=str(srcnode),label=str(node_index_id2msg_mal[srcnode]), color='purple',shape = 'box')
            if "'subject': '" in temp_edge['srcmsg']:
                src_shape='box'
            elif "'file': '" in temp_edge['srcmsg']:
                src_shape='oval'
            elif "'netflow': '" in temp_edge['srcmsg']:
                src_shape='diamond'
                
            src_shape='box'
            if attack_edge_flag(temp_edge['srcmsg']):
                src_node_color='red'
                total_nodes+=1
            else:
                src_node_color='blue'
                
            src_node_color='blue'
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['srcmsg']))),
                     label=str(replace_path_name(temp_edge['srcmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['srcmsg'])))])), 
                     color=src_node_color,
                     shape = src_shape)


            if "'subject': '" in temp_edge['dstmsg']:
                dst_shape='box'
            elif "'file': '" in temp_edge['dstmsg']:
                dst_shape='oval'
            elif "'netflow': '" in temp_edge['dstmsg']:
                dst_shape='diamond'

                    
            if "->" in temp_edge['dstmsg']:
                dst_shape='diamond'
            else:
                dst_shape='oval'
                
                
            if attack_edge_flag(temp_edge['dstmsg']):
                dst_node_color='red'
                total_nodes+=1
            else:
                dst_node_color='blue'
            dst_node_color='blue'
            
            
            dot.node( name=str(hashgen(replace_path_name(temp_edge['dstmsg']))),
                     label=str(replace_path_name(temp_edge['dstmsg'])+'\t partition:'+str(partition[str(hashgen(replace_path_name(temp_edge['dstmsg'])))])), 
                     color=dst_node_color,
                     shape = dst_shape)


    #         edgeindex=tensor_find(test_data.msg[i][16:-16],1)
    
            temp_edge_visual=replace_path_name(temp_edge['srcmsg'])+','+replace_path_name(temp_edge['dstmsg'])+','+str(temp_edge['time'])
            temp_edge_hash_val=hashgen(temp_edge_visual)
            
            if temp_edge_hash_val in edges_attack_hashset_h051:
                edge_color='red'
                attck_nodes_set_visual.add(replace_path_name(temp_edge['srcmsg']))
                attck_nodes_set_visual.add(replace_path_name(temp_edge['dstmsg']))
                attack_edge_count+=1
            else:
                edge_color='blue'
            
#             if attack_edge_flag(temp_edge['srcmsg']) and attack_edge_flag(temp_edge['dstmsg']):
#                 edge_color='red'
#                 attack_edge_count+=1
#             else:
#                 edge_color='blue'
                
                
            dot.edge(str(hashgen(replace_path_name(temp_edge['srcmsg']))),str(hashgen(replace_path_name(temp_edge['dstmsg']))), label= temp_edge['edge_type'] , color=edge_color)#+ "  loss: "+str(temp_edge['loss']) + "  time: "+str(temp_edge['time'])

    #         dot.edge(str(srcnode), str(dstnode), label= temp_edge['edge_type']+ "  loss: "+str((temp_edge['loss'])) + "  time: "+str(temp_edge['time']) , color='red')


#     if len(communities[c].edges)<2:
#         continue
#     if len(communities[c].edges)>1000:
#         continue
#         print(f"edge num:{len(communities[c].edges)}，skip rendering")
        
    print("Start to render the figures···")
    
    dot.render('./graph_visual_h051/subgraph_'+str(graph_index), view=False)
    print("subgraph loss:",(subgraph_loss_sum/len(communities[c].edges)))
    print("graph_index:",graph_index)
    print("edge_count:",len(communities[c].edges))
    print("node_count:",len(communities[c].nodes))
    print(f"{attack_node_count=}")
    print(f"{attack_edge_count=}")
    graph_index+=1

print(f"avg edges:{total_edges/len(communities)}")
print(f"avg nodes:{total_nodes/len(communities)}")